In [37]:
from __future__ import division

import os
import random
import multiprocessing as mp
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.path as mplpath
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from scipy.spatial.distance import cosine, cdist

from learning2learn.wrangle import (synthesize_data, get_train_test_parameters, 
                                    build_train_set, build_test_trials_order1,
                                    build_test_trials_order2)
from learning2learn.util import get_hidden_representations, train_test_split, evaluate_generalization, train_model
from learning2learn.images import generate_image
from learning2learn.models import build_model

In [2]:
gpu_options = tf.GPUOptions(allow_growth=True, visible_device_list='1')
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
K.set_session(sess)

In [146]:
def get_data(nb_categories, nb_exemplars):
    np.random.seed(0)
    df_train, labels = synthesize_data(nb_categories, nb_exemplars)
    labels = labels.values
    (shape_set_train, shape_set_test), \
    (color_set_train, color_set_test), \
    (texture_set_train, texture_set_test) = \
        get_train_test_parameters()
    if nb_categories < 50:
        shape_set_train, _ = train_test_split(shape_set_train, 50-nb_categories)
        color_set_train, _ = train_test_split(color_set_train, 50-nb_categories)
        texture_set_train, _ = train_test_split(texture_set_train, 50-nb_categories)
    X_train = build_train_set(
        df_train, shape_set_train, color_set_train,
        texture_set_train,
        target_size=(200, 200)
    )
    ohe = OneHotEncoder(sparse=False)
    Y_train = ohe.fit_transform(labels.reshape(-1, 1))
    X_test_o1 = build_test_trials_order1(
        df_train, shape_set_train, shape_set_test, color_set_train,
        color_set_test, texture_set_train, texture_set_test,
        nb_trials=1000
    )
    X_test_o2 = build_test_trials_order2(
        shape_set_test, color_set_test, texture_set_test,
        nb_trials=1000
    )
    
    return X_train, Y_train, X_test_o1, X_test_o2

In [107]:
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.regularizers import l2

def cnn0(input_shape, nb_classes):
    # The baseline CNN
    layers = [
        # Conv, Pool
        Conv2D(32, (5, 5), padding='same', input_shape=input_shape),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Conv, Pool
        Conv2D(64, (5, 5), padding='same'),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Flatten
        Flatten(),
        # Hidden layer
        Dropout(0.5),
        Dense(2048),
        Activation('relu'),
        # Output layer
        Dropout(0.5),
        Dense(nb_classes),
        Activation('softmax')
    ]

    return build_model(layers)

def cnn1(input_shape, nb_classes):
    # Baseline but with dropout removed from last pooling layer
    layers = [
        # Conv, Pool
        Conv2D(32, (5, 5), padding='same', input_shape=input_shape),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Conv, Pool
        Conv2D(64, (5, 5), padding='same'),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Flatten
        Flatten(),
        # Hidden layer
        Dense(2048),
        Activation('relu'),
        # Output layer
        Dropout(0.5),
        Dense(nb_classes),
        Activation('softmax')
    ]

    return build_model(layers)

def cnn2(input_shape, nb_classes):
    # Baseline but with regularization added
    layers = [
        # Conv, Pool
        Conv2D(32, (5, 5), padding='same', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Conv, Pool
        Conv2D(64, (5, 5), padding='same', kernel_regularizer=l2(0.01)),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Flatten
        Flatten(),
        # Hidden layer
        Dropout(0.5),
        Dense(2048),
        Activation('relu'),
        # Output layer
        Dropout(0.5),
        Dense(nb_classes),
        Activation('softmax')
    ]

    return build_model(layers)

def cnn3(input_shape, nb_classes):
    # Much smaller filters and FC layer, regularization added
    layers = [
        # Conv, Pool
        Conv2D(5, (5, 5), padding='same', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Conv, Pool
        Conv2D(5, (5, 5), padding='same', kernel_regularizer=l2(0.01)),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Flatten
        Flatten(),
        # Hidden layer
        Dropout(0.5),
        Dense(100),
        Activation('relu'),
        # Output layer
        Dropout(0.5),
        Dense(nb_classes),
        Activation('softmax')
    ]

    return build_model(layers)

def cnn4(input_shape, nb_classes):
    # Much smaller filters and FC layer, regularization added, dropout 
    # removed after last pool layer
    layers = [
        # Conv, Pool
        Conv2D(5, (5, 5), padding='same', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Conv, Pool
        Conv2D(5, (5, 5), padding='same', kernel_regularizer=l2(0.01)),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Flatten
        Flatten(),
        # Hidden layer
        Dense(100),
        Activation('relu'),
        # Output layer
        Dropout(0.5),
        Dense(nb_classes),
        Activation('softmax')
    ]

    return build_model(layers)

def cnn5(input_shape, nb_classes):
    # Much smaller filters and FC layer, regularization added, dropout 
    # removed after last pool layer, 25 units in FC
    layers = [
        # Conv, Pool
        Conv2D(5, (5, 5), padding='same', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Conv, Pool
        Conv2D(5, (5, 5), padding='same', kernel_regularizer=l2(0.01)),
        Activation('relu'),
        MaxPooling2D(pool_size=(5, 5)),
        # Flatten
        Flatten(),
        # Hidden layer
        Dense(25),
        Activation('relu'),
        # Output layer
        Dropout(0.5),
        Dense(nb_classes),
        Activation('softmax')
    ]

    return build_model(layers)

In [117]:
def model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_runs=5, nb_epoch=100, burn_period=20
):
    scores_o1 = np.zeros(nb_runs)
    scores_o2 = np.zeros(nb_runs)
    for i in range(nb_runs):
        print('Trial #%i' % (i+1))
        weights_file = '../data/temp.h5'
        if os.path.isfile(weights_file):
            os.remove(weights_file)
        checkpoint = ModelCheckpoint(
            weights_file,
            monitor='loss',
            save_best_only=True,
            save_weights_only=True,
            period=2
        )
        train_model(
            model, X_train, Y_train, epochs=nb_epoch,
            validation_data=None, batch_size=32,
            checkpoint=checkpoint, burn_period=burn_period
        )
        model.load_weights(weights_file)
        score_o1 = evaluate_generalization(
            model, X_test_o1, layer_num=-4,
            batch_size=128
        )
        score_o2 = evaluate_generalization(
            model, X_test_o2, layer_num=-4,
            batch_size=128
        )
        print('\nScore O1: %0.4f' % score_o1)
        print('Score O2: %0.4f\n' % score_o2)
        scores_o1[i] = score_o1
        scores_o2[i] = score_o2
    
    print('\nScore O1 avg: %0.4f' % scores_o1.mean())
    print('Score O2 avg: %0.4f' % scores_o2.mean())
    return scores_o1, scores_o2

In [118]:
nb_categories = 10
nb_exemplars = 5
X_train, Y_train, X_test_o1, X_test_o2 = get_data(nb_categories, nb_exemplars)

In [ ]:
# baseline

In [51]:
model = cnn0(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2
)

Epoch 1/20
50/50 [==============================] - 0s 9ms/step - loss: 3.7450 - acc: 0.1000
Epoch 2/20
50/50 [==============================] - 0s 3ms/step - loss: 2.4595 - acc: 0.1600
Epoch 3/20
50/50 [==============================] - 0s 3ms/step - loss: 2.2877 - acc: 0.0800
Epoch 4/20
50/50 [==============================] - 0s 3ms/step - loss: 2.2359 - acc: 0.1800
Epoch 5/20
50/50 [==============================] - 0s 3ms/step - loss: 2.2014 - acc: 0.1800
Epoch 6/20
50/50 [==============================] - 0s 3ms/step - loss: 2.1612 - acc: 0.1800
Epoch 7/20
50/50 [==============================] - 0s 3ms/step - loss: 2.0521 - acc: 0.2200
Epoch 8/20
50/50 [==============================] - 0s 3ms/step - loss: 1.9081 - acc: 0.4000
Epoch 9/20
50/50 [==============================] - 0s 3ms/step - loss: 1.8220 - acc: 0.3600
Epoch 10/20
50/50 [==============================] - 0s 3ms/step - loss: 1.9541 - acc: 0.2400
Epoch 11/20
50/50 [==============================] - 0s 3ms/step - lo

50/50 [==============================] - 0s 3ms/step - loss: 1.3077 - acc: 0.7200
Epoch 68/80
50/50 [==============================] - 0s 3ms/step - loss: 0.1082 - acc: 0.9800
Epoch 69/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0172 - acc: 1.0000
Epoch 70/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0074 - acc: 1.0000
Epoch 71/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0158 - acc: 1.0000
Epoch 72/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0126 - acc: 1.0000
Epoch 73/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0029 - acc: 1.0000
Epoch 74/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0025 - acc: 1.0000
Epoch 75/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0028 - acc: 1.0000
Epoch 76/80
50/50 [==============================] - 0s 3ms/step - loss: 3.8808e-04 - acc: 1.0000
Epoch 77/80
50/50 [==============================] - 0s 3ms/step - l

50/50 [==============================] - 0s 3ms/step - loss: 3.4111e-05 - acc: 1.0000
Epoch 53/80
50/50 [==============================] - 0s 3ms/step - loss: 1.6984e-04 - acc: 1.0000
Epoch 54/80
50/50 [==============================] - 0s 3ms/step - loss: 7.3447e-05 - acc: 1.0000
Epoch 55/80
50/50 [==============================] - 0s 3ms/step - loss: 8.9712e-05 - acc: 1.0000
Epoch 56/80
50/50 [==============================] - 0s 3ms/step - loss: 3.7640e-05 - acc: 1.0000
Epoch 57/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0012 - acc: 1.0000
Epoch 58/80
50/50 [==============================] - 0s 3ms/step - loss: 2.2330e-04 - acc: 1.0000
Epoch 59/80
50/50 [==============================] - 0s 3ms/step - loss: 2.0738e-04 - acc: 1.0000
Epoch 60/80
50/50 [==============================] - 1s 13ms/step - loss: 1.5385e-05 - acc: 1.0000
Epoch 61/80
50/50 [==============================] - 0s 3ms/step - loss: 2.1619e-05 - acc: 1.0000
Epoch 62/80
50/50 [================

50/50 [==============================] - 0s 3ms/step - loss: 1.4451e-04 - acc: 1.0000
Epoch 37/80
50/50 [==============================] - 0s 3ms/step - loss: 2.9399e-04 - acc: 1.0000
Epoch 38/80
50/50 [==============================] - 0s 3ms/step - loss: 4.2929e-04 - acc: 1.0000
Epoch 39/80
50/50 [==============================] - 0s 3ms/step - loss: 9.3005e-05 - acc: 1.0000
Epoch 40/80
50/50 [==============================] - 0s 3ms/step - loss: 2.5465e-04 - acc: 1.0000
Epoch 41/80
50/50 [==============================] - 0s 3ms/step - loss: 6.8331e-05 - acc: 1.0000
Epoch 42/80
50/50 [==============================] - 0s 3ms/step - loss: 4.1007e-05 - acc: 1.0000
Epoch 43/80
50/50 [==============================] - 0s 3ms/step - loss: 2.1167e-04 - acc: 1.0000
Epoch 44/80
50/50 [==============================] - 0s 3ms/step - loss: 1.4234e-04 - acc: 1.0000
Epoch 45/80
50/50 [==============================] - 0s 3ms/step - loss: 8.6611e-05 - acc: 1.0000
Epoch 46/80
50/50 [=============

50/50 [==============================] - 0s 3ms/step - loss: 1.7512e-06 - acc: 1.0000
Epoch 20/80
50/50 [==============================] - 1s 14ms/step - loss: 7.6175e-07 - acc: 1.0000
Epoch 21/80
50/50 [==============================] - 0s 3ms/step - loss: 2.2757e-06 - acc: 1.0000
Epoch 22/80
50/50 [==============================] - 0s 3ms/step - loss: 2.6358e-06 - acc: 1.0000
Epoch 23/80
50/50 [==============================] - 0s 3ms/step - loss: 6.6654e-06 - acc: 1.0000
Epoch 24/80
50/50 [==============================] - 0s 3ms/step - loss: 1.1216e-04 - acc: 1.0000
Epoch 25/80
50/50 [==============================] - 0s 3ms/step - loss: 4.5947e-06 - acc: 1.0000
Epoch 26/80
50/50 [==============================] - 0s 3ms/step - loss: 1.7935e-05 - acc: 1.0000
Epoch 27/80
50/50 [==============================] - 0s 3ms/step - loss: 1.4901e-07 - acc: 1.0000
Epoch 28/80
50/50 [==============================] - 1s 13ms/step - loss: 5.6267e-07 - acc: 1.0000
Epoch 29/80
50/50 [===========

50/50 [==============================] - 1s 15ms/step - loss: 8.1182e-07 - acc: 1.0000
Epoch 3/80
50/50 [==============================] - 0s 3ms/step - loss: 6.3181e-07 - acc: 1.0000
Epoch 4/80
50/50 [==============================] - 1s 14ms/step - loss: 2.8729e-07 - acc: 1.0000
Epoch 5/80
50/50 [==============================] - 0s 3ms/step - loss: 4.9114e-07 - acc: 1.0000
Epoch 6/80
50/50 [==============================] - 0s 3ms/step - loss: 2.5166e-06 - acc: 1.0000
Epoch 7/80
50/50 [==============================] - 0s 3ms/step - loss: 9.6919e-07 - acc: 1.0000
Epoch 8/80
50/50 [==============================] - 1s 15ms/step - loss: 1.3590e-07 - acc: 1.0000
Epoch 9/80
50/50 [==============================] - 0s 3ms/step - loss: 6.2943e-07 - acc: 1.0000
Epoch 10/80
50/50 [==============================] - 0s 3ms/step - loss: 9.2562e-06 - acc: 1.0000
Epoch 11/80
50/50 [==============================] - 0s 3ms/step - loss: 7.4149e-07 - acc: 1.0000
Epoch 12/80
50/50 [=================

In [52]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.6138
Score O2 avg: 0.4580


In [ ]:
# model 1

In [54]:
model = cnn1(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2
)

Epoch 1/20
50/50 [==============================] - 0s 9ms/step - loss: 6.4314 - acc: 0.1000
Epoch 2/20
50/50 [==============================] - 0s 3ms/step - loss: 5.6907 - acc: 0.0400
Epoch 3/20
50/50 [==============================] - 0s 3ms/step - loss: 2.2962 - acc: 0.1000
Epoch 4/20
50/50 [==============================] - 0s 3ms/step - loss: 2.2410 - acc: 0.2000
Epoch 5/20
50/50 [==============================] - 0s 3ms/step - loss: 2.1804 - acc: 0.1400
Epoch 6/20
50/50 [==============================] - 0s 3ms/step - loss: 2.1154 - acc: 0.1800
Epoch 7/20
50/50 [==============================] - 0s 3ms/step - loss: 2.0404 - acc: 0.2600
Epoch 8/20
50/50 [==============================] - 0s 3ms/step - loss: 1.9357 - acc: 0.3400
Epoch 9/20
50/50 [==============================] - 0s 3ms/step - loss: 2.1271 - acc: 0.2400
Epoch 10/20
50/50 [==============================] - 0s 3ms/step - loss: 1.8385 - acc: 0.4400
Epoch 11/20
50/50 [==============================] - 0s 3ms/step - lo

50/50 [==============================] - 0s 3ms/step - loss: 3.8340e-05 - acc: 1.0000
Epoch 68/80
50/50 [==============================] - 0s 3ms/step - loss: 1.3619e-04 - acc: 1.0000
Epoch 69/80
50/50 [==============================] - 0s 3ms/step - loss: 3.1372e-04 - acc: 1.0000
Epoch 70/80
50/50 [==============================] - 1s 13ms/step - loss: 5.2573e-05 - acc: 1.0000
Epoch 71/80
50/50 [==============================] - 0s 3ms/step - loss: 4.3951e-05 - acc: 1.0000
Epoch 72/80
50/50 [==============================] - 0s 3ms/step - loss: 9.2729e-05 - acc: 1.0000
Epoch 73/80
50/50 [==============================] - 0s 3ms/step - loss: 6.8385e-05 - acc: 1.0000
Epoch 74/80
50/50 [==============================] - 1s 28ms/step - loss: 3.8015e-05 - acc: 1.0000
Epoch 75/80
50/50 [==============================] - 0s 3ms/step - loss: 1.8289e-04 - acc: 1.0000
Epoch 76/80
50/50 [==============================] - 0s 3ms/step - loss: 0.8675 - acc: 0.8400
Epoch 77/80
50/50 [===============

50/50 [==============================] - 0s 3ms/step - loss: 1.5803e-05 - acc: 1.0000
Epoch 52/80
50/50 [==============================] - 0s 3ms/step - loss: 4.4568e-05 - acc: 1.0000
Epoch 53/80
50/50 [==============================] - 0s 3ms/step - loss: 2.4215e-05 - acc: 1.0000
Epoch 54/80
50/50 [==============================] - 0s 3ms/step - loss: 2.2287e-05 - acc: 1.0000
Epoch 55/80
50/50 [==============================] - 0s 3ms/step - loss: 4.1569e-06 - acc: 1.0000
Epoch 56/80
50/50 [==============================] - 1s 13ms/step - loss: 2.6524e-06 - acc: 1.0000
Epoch 57/80
50/50 [==============================] - 0s 3ms/step - loss: 3.9697e-06 - acc: 1.0000
Epoch 58/80
50/50 [==============================] - 0s 3ms/step - loss: 3.3177e-06 - acc: 1.0000
Epoch 59/80
50/50 [==============================] - 0s 3ms/step - loss: 3.0077e-06 - acc: 1.0000
Epoch 60/80
50/50 [==============================] - 1s 13ms/step - loss: 2.2269e-06 - acc: 1.0000
Epoch 61/80
50/50 [===========

50/50 [==============================] - 1s 13ms/step - loss: 3.7540e-06 - acc: 1.0000
Epoch 35/80
50/50 [==============================] - 0s 3ms/step - loss: 6.3088e-06 - acc: 1.0000
Epoch 36/80
50/50 [==============================] - 0s 3ms/step - loss: 5.3467e-06 - acc: 1.0000
Epoch 37/80
50/50 [==============================] - 0s 3ms/step - loss: 3.6420e-05 - acc: 1.0000
Epoch 38/80
50/50 [==============================] - 0s 3ms/step - loss: 4.5624e-06 - acc: 1.0000
Epoch 39/80
50/50 [==============================] - 0s 3ms/step - loss: 2.9183e-06 - acc: 1.0000
Epoch 40/80
50/50 [==============================] - 1s 13ms/step - loss: 2.5082e-06 - acc: 1.0000
Epoch 41/80
50/50 [==============================] - 0s 3ms/step - loss: 1.7536e-06 - acc: 1.0000
Epoch 42/80
50/50 [==============================] - 0s 3ms/step - loss: 1.3509e-05 - acc: 1.0000
Epoch 43/80
50/50 [==============================] - 0s 3ms/step - loss: 9.2181e-04 - acc: 1.0000
Epoch 44/80
50/50 [===========

50/50 [==============================] - 0s 3ms/step - loss: 1.4663e-07 - acc: 1.0000
Epoch 18/80
50/50 [==============================] - 0s 3ms/step - loss: 1.9908e-07 - acc: 1.0000
Epoch 19/80
50/50 [==============================] - 0s 3ms/step - loss: 1.5616e-07 - acc: 1.0000
Epoch 20/80
50/50 [==============================] - 0s 3ms/step - loss: 5.0545e-07 - acc: 1.0000
Epoch 21/80
50/50 [==============================] - 0s 3ms/step - loss: 1.7166e-07 - acc: 1.0000
Epoch 22/80
50/50 [==============================] - 1s 13ms/step - loss: 1.3828e-07 - acc: 1.0000
Epoch 23/80
50/50 [==============================] - 0s 3ms/step - loss: 1.5497e-07 - acc: 1.0000
Epoch 24/80
50/50 [==============================] - 0s 3ms/step - loss: 1.4782e-07 - acc: 1.0000
Epoch 25/80
50/50 [==============================] - 0s 3ms/step - loss: 7.1169e-07 - acc: 1.0000
Epoch 26/80
50/50 [==============================] - 0s 3ms/step - loss: 3.3379e-07 - acc: 1.0000
Epoch 27/80
50/50 [============

50/50 [==============================] - 0s 3ms/step - loss: 2.0504e-07 - acc: 1.0000
Epoch 1/80
50/50 [==============================] - 0s 3ms/step - loss: 1.9550e-07 - acc: 1.0000
Epoch 2/80
50/50 [==============================] - 1s 13ms/step - loss: 1.6093e-07 - acc: 1.0000
Epoch 3/80
50/50 [==============================] - 0s 3ms/step - loss: 1.4186e-07 - acc: 1.0000
Epoch 4/80
50/50 [==============================] - 1s 13ms/step - loss: 1.4067e-07 - acc: 1.0000
Epoch 5/80
50/50 [==============================] - 0s 3ms/step - loss: 1.5497e-07 - acc: 1.0000
Epoch 6/80
50/50 [==============================] - 0s 3ms/step - loss: 2.3246e-07 - acc: 1.0000
Epoch 7/80
50/50 [==============================] - 0s 3ms/step - loss: 1.7881e-07 - acc: 1.0000
Epoch 8/80
50/50 [==============================] - 0s 3ms/step - loss: 2.7657e-07 - acc: 1.0000
Epoch 9/80
50/50 [==============================] - 0s 3ms/step - loss: 1.3590e-07 - acc: 1.0000
Epoch 10/80
50/50 [====================

In [55]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.5726
Score O2 avg: 0.3820


In [ ]:
# model 2

In [57]:
model = cnn2(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2
)

Epoch 1/20
50/50 [==============================] - 1s 13ms/step - loss: 4.5483 - acc: 0.0600
Epoch 2/20
50/50 [==============================] - 0s 3ms/step - loss: 3.0499 - acc: 0.0400
Epoch 3/20
50/50 [==============================] - 0s 3ms/step - loss: 2.6804 - acc: 0.1800
Epoch 4/20
50/50 [==============================] - 0s 3ms/step - loss: 2.6562 - acc: 0.2000
Epoch 5/20
50/50 [==============================] - 0s 3ms/step - loss: 2.5521 - acc: 0.2000
Epoch 6/20
50/50 [==============================] - 0s 3ms/step - loss: 2.4357 - acc: 0.3400
Epoch 7/20
50/50 [==============================] - 0s 3ms/step - loss: 2.2526 - acc: 0.3400
Epoch 8/20
50/50 [==============================] - 0s 3ms/step - loss: 2.0656 - acc: 0.4400
Epoch 9/20
50/50 [==============================] - 0s 3ms/step - loss: 2.3266 - acc: 0.3400
Epoch 10/20
50/50 [==============================] - 0s 3ms/step - loss: 2.1988 - acc: 0.3400
Epoch 11/20
50/50 [==============================] - 0s 3ms/step - l

50/50 [==============================] - 0s 3ms/step - loss: 0.3138 - acc: 0.9200
Epoch 69/80
50/50 [==============================] - 0s 3ms/step - loss: 0.1756 - acc: 0.9800
Epoch 70/80
50/50 [==============================] - 1s 13ms/step - loss: 0.1387 - acc: 1.0000
Epoch 71/80
50/50 [==============================] - 0s 3ms/step - loss: 0.1379 - acc: 1.0000
Epoch 72/80
50/50 [==============================] - 0s 3ms/step - loss: 0.1445 - acc: 1.0000
Epoch 73/80
50/50 [==============================] - 0s 3ms/step - loss: 0.2344 - acc: 0.9400
Epoch 74/80
50/50 [==============================] - 0s 3ms/step - loss: 0.1589 - acc: 1.0000
Epoch 75/80
50/50 [==============================] - 0s 3ms/step - loss: 0.1331 - acc: 1.0000
Epoch 76/80
50/50 [==============================] - 1s 13ms/step - loss: 0.1313 - acc: 1.0000
Epoch 77/80
50/50 [==============================] - 0s 3ms/step - loss: 0.1287 - acc: 1.0000
Epoch 78/80
50/50 [==============================] - 1s 14ms/step - lo

50/50 [==============================] - 0s 3ms/step - loss: 0.0614 - acc: 1.0000
Epoch 55/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0602 - acc: 1.0000
Epoch 56/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0610 - acc: 1.0000
Epoch 57/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0561 - acc: 1.0000
Epoch 58/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0595 - acc: 1.0000
Epoch 59/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0646 - acc: 1.0000
Epoch 60/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0542 - acc: 1.0000
Epoch 61/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0532 - acc: 1.0000
Epoch 62/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0534 - acc: 1.0000
Epoch 63/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0524 - acc: 1.0000
Epoch 64/80
50/50 [==============================] - 0s 3ms/step - loss:

Epoch 41/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0379 - acc: 1.0000
Epoch 42/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0478 - acc: 1.0000
Epoch 43/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0464 - acc: 1.0000
Epoch 44/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0397 - acc: 1.0000
Epoch 45/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0361 - acc: 1.0000
Epoch 46/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0360 - acc: 1.0000
Epoch 47/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0355 - acc: 1.0000
Epoch 48/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0357 - acc: 1.0000
Epoch 49/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0346 - acc: 1.0000
Epoch 50/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0349 - acc: 1.0000
Epoch 51/80
50/50 [==============================] - 0s 3ms/

50/50 [==============================] - 0s 3ms/step - loss: 0.0228 - acc: 1.0000
Epoch 28/80
50/50 [==============================] - 1s 15ms/step - loss: 0.0222 - acc: 1.0000
Epoch 29/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0218 - acc: 1.0000
Epoch 30/80
50/50 [==============================] - 1s 15ms/step - loss: 0.0214 - acc: 1.0000
Epoch 31/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0216 - acc: 1.0000
Epoch 32/80
50/50 [==============================] - 1s 15ms/step - loss: 0.0212 - acc: 1.0000
Epoch 33/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0203 - acc: 1.0000
Epoch 34/80
50/50 [==============================] - 1s 14ms/step - loss: 0.0205 - acc: 1.0000
Epoch 35/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0195 - acc: 1.0000
Epoch 36/80
50/50 [==============================] - 1s 14ms/step - loss: 0.0192 - acc: 1.0000
Epoch 37/80
50/50 [==============================] - 0s 3ms/step - 

Epoch 14/80
50/50 [==============================] - 1s 14ms/step - loss: 0.0164 - acc: 1.0000
Epoch 15/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0160 - acc: 1.0000
Epoch 16/80
50/50 [==============================] - 1s 14ms/step - loss: 0.0158 - acc: 1.0000
Epoch 17/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0778 - acc: 0.9800
Epoch 18/80
50/50 [==============================] - 0s 3ms/step - loss: 0.4724 - acc: 0.8200
Epoch 19/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0869 - acc: 0.9800
Epoch 20/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0672 - acc: 0.9800
Epoch 21/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0182 - acc: 1.0000
Epoch 22/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0181 - acc: 1.0000
Epoch 23/80
50/50 [==============================] - 0s 3ms/step - loss: 0.0182 - acc: 1.0000
Epoch 24/80
50/50 [==============================] - 0s 3m

In [58]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.7214
Score O2 avg: 0.5392


In [ ]:
# First make sure model 3 training loss goes down...

In [69]:
model = cnn3(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
model.fit(
    X_train, Y_train, epochs=400,
    validation_data=None, batch_size=32,
    shuffle=True, verbose=1
)

Epoch 1/400
50/50 [==============================] - 1s 13ms/step - loss: 4.1146 - acc: 0.0800
Epoch 2/400
50/50 [==============================] - 0s 1ms/step - loss: 2.5839 - acc: 0.1200
Epoch 3/400
50/50 [==============================] - 0s 1ms/step - loss: 2.6716 - acc: 0.0600
Epoch 4/400
50/50 [==============================] - 0s 1ms/step - loss: 2.3711 - acc: 0.2200
Epoch 5/400
50/50 [==============================] - 0s 2ms/step - loss: 2.5654 - acc: 0.0800
Epoch 6/400
50/50 [==============================] - 0s 2ms/step - loss: 2.4049 - acc: 0.0600
Epoch 7/400
50/50 [==============================] - 0s 2ms/step - loss: 2.4738 - acc: 0.0400
Epoch 8/400
50/50 [==============================] - 0s 2ms/step - loss: 2.4044 - acc: 0.0600
Epoch 9/400
50/50 [==============================] - 0s 2ms/step - loss: 2.4291 - acc: 0.1000
Epoch 10/400
50/50 [==============================] - 0s 1ms/step - loss: 2.3511 - acc: 0.1200
Epoch 11/400
50/50 [==============================] - 0s 1

50/50 [==============================] - 0s 2ms/step - loss: 1.0039 - acc: 0.6400
Epoch 88/400
50/50 [==============================] - 0s 2ms/step - loss: 0.7980 - acc: 0.8400
Epoch 89/400
50/50 [==============================] - 0s 1ms/step - loss: 1.1234 - acc: 0.6600
Epoch 90/400
50/50 [==============================] - 0s 1ms/step - loss: 1.1706 - acc: 0.5800
Epoch 91/400
50/50 [==============================] - 0s 2ms/step - loss: 1.1125 - acc: 0.7200
Epoch 92/400
50/50 [==============================] - 0s 1ms/step - loss: 1.0434 - acc: 0.6800
Epoch 93/400
50/50 [==============================] - 0s 1ms/step - loss: 1.0217 - acc: 0.6400
Epoch 94/400
50/50 [==============================] - 0s 1ms/step - loss: 0.9054 - acc: 0.7200
Epoch 95/400
50/50 [==============================] - 0s 2ms/step - loss: 1.0800 - acc: 0.6800
Epoch 96/400
50/50 [==============================] - 0s 1ms/step - loss: 0.8387 - acc: 0.7800
Epoch 97/400
50/50 [==============================] - 0s 2ms/st

50/50 [==============================] - 0s 2ms/step - loss: 0.3633 - acc: 0.9000
Epoch 173/400
50/50 [==============================] - 0s 2ms/step - loss: 0.3743 - acc: 0.9400
Epoch 174/400
50/50 [==============================] - 0s 2ms/step - loss: 0.4193 - acc: 0.8600
Epoch 175/400
50/50 [==============================] - 0s 2ms/step - loss: 0.4487 - acc: 0.8400
Epoch 176/400
50/50 [==============================] - 0s 2ms/step - loss: 0.4547 - acc: 0.9600
Epoch 177/400
50/50 [==============================] - 0s 2ms/step - loss: 0.4782 - acc: 0.8400
Epoch 178/400
50/50 [==============================] - 0s 2ms/step - loss: 0.3994 - acc: 0.8800
Epoch 179/400
50/50 [==============================] - 0s 2ms/step - loss: 0.5153 - acc: 0.8600
Epoch 180/400
50/50 [==============================] - 0s 2ms/step - loss: 0.4842 - acc: 0.8400
Epoch 181/400
50/50 [==============================] - 0s 2ms/step - loss: 0.4006 - acc: 0.9200
Epoch 182/400
50/50 [==============================] -

50/50 [==============================] - 0s 1ms/step - loss: 0.2953 - acc: 0.9200
Epoch 258/400
50/50 [==============================] - 0s 1ms/step - loss: 0.2504 - acc: 0.9400
Epoch 259/400
50/50 [==============================] - 0s 1ms/step - loss: 0.2388 - acc: 0.9800
Epoch 260/400
50/50 [==============================] - 0s 2ms/step - loss: 0.3197 - acc: 0.9000
Epoch 261/400
50/50 [==============================] - 0s 1ms/step - loss: 0.3028 - acc: 0.9200
Epoch 262/400
50/50 [==============================] - 0s 2ms/step - loss: 0.3282 - acc: 0.9200
Epoch 263/400
50/50 [==============================] - 0s 2ms/step - loss: 0.3959 - acc: 0.9200
Epoch 264/400
50/50 [==============================] - 0s 2ms/step - loss: 0.3331 - acc: 0.8800
Epoch 265/400
50/50 [==============================] - 0s 1ms/step - loss: 0.3411 - acc: 0.9400
Epoch 266/400
50/50 [==============================] - 0s 1ms/step - loss: 0.2731 - acc: 0.9400
Epoch 267/400
50/50 [==============================] -

50/50 [==============================] - 0s 1ms/step - loss: 0.1839 - acc: 0.9800
Epoch 343/400
50/50 [==============================] - 0s 2ms/step - loss: 0.2032 - acc: 0.9600
Epoch 344/400
50/50 [==============================] - 0s 2ms/step - loss: 0.2468 - acc: 0.9200
Epoch 345/400
50/50 [==============================] - 0s 2ms/step - loss: 0.2333 - acc: 0.9600
Epoch 346/400
50/50 [==============================] - 0s 2ms/step - loss: 0.1951 - acc: 0.9400
Epoch 347/400
50/50 [==============================] - 0s 1ms/step - loss: 0.1516 - acc: 1.0000
Epoch 348/400
50/50 [==============================] - 0s 2ms/step - loss: 0.2567 - acc: 0.9400
Epoch 349/400
50/50 [==============================] - 0s 1ms/step - loss: 0.2511 - acc: 0.9400
Epoch 350/400
50/50 [==============================] - 0s 1ms/step - loss: 0.3983 - acc: 0.9200
Epoch 351/400
50/50 [==============================] - 0s 1ms/step - loss: 0.2774 - acc: 0.9400
Epoch 352/400
50/50 [==============================] -

In [ ]:
# Now evaluate with 400 training epochs per trial

In [77]:
model = cnn3(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=400, burn_period=100
)

Epoch 1/100
50/50 [==============================] - 1s 15ms/step - loss: 2.4653 - acc: 0.1800
Epoch 2/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3398 - acc: 0.1400
Epoch 3/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3837 - acc: 0.1600
Epoch 4/100
50/50 [==============================] - 0s 1ms/step - loss: 2.4197 - acc: 0.1000
Epoch 5/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3198 - acc: 0.1600
Epoch 6/100
50/50 [==============================] - 0s 1ms/step - loss: 2.4434 - acc: 0.0400
Epoch 7/100
50/50 [==============================] - 0s 2ms/step - loss: 2.4011 - acc: 0.0800
Epoch 8/100
50/50 [==============================] - 0s 1ms/step - loss: 2.3635 - acc: 0.2000
Epoch 9/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3219 - acc: 0.1200
Epoch 10/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3261 - acc: 0.2400
Epoch 11/100
50/50 [==============================] - 0s 2

50/50 [==============================] - 0s 2ms/step - loss: 0.7274 - acc: 0.8800
Epoch 88/100
50/50 [==============================] - 0s 1ms/step - loss: 0.7709 - acc: 0.8200
Epoch 89/100
50/50 [==============================] - 0s 2ms/step - loss: 0.7489 - acc: 0.7400
Epoch 90/100
50/50 [==============================] - 0s 2ms/step - loss: 0.9797 - acc: 0.7400
Epoch 91/100
50/50 [==============================] - 0s 2ms/step - loss: 0.6822 - acc: 0.8000
Epoch 92/100
50/50 [==============================] - 0s 1ms/step - loss: 0.8607 - acc: 0.7800
Epoch 93/100
50/50 [==============================] - 0s 1ms/step - loss: 0.6938 - acc: 0.8000
Epoch 94/100
50/50 [==============================] - 0s 1ms/step - loss: 0.7093 - acc: 0.7800
Epoch 95/100
50/50 [==============================] - 0s 1ms/step - loss: 0.5993 - acc: 0.8400
Epoch 96/100
50/50 [==============================] - 0s 2ms/step - loss: 0.7809 - acc: 0.8200
Epoch 97/100
50/50 [==============================] - 0s 1ms/st

50/50 [==============================] - 0s 1ms/step - loss: 0.4023 - acc: 0.8800
Epoch 74/300
50/50 [==============================] - 0s 2ms/step - loss: 0.3073 - acc: 0.9600
Epoch 75/300
50/50 [==============================] - 0s 2ms/step - loss: 0.3837 - acc: 0.9200
Epoch 76/300
50/50 [==============================] - 0s 2ms/step - loss: 0.2629 - acc: 0.9600
Epoch 77/300
50/50 [==============================] - 0s 2ms/step - loss: 0.4242 - acc: 0.8800
Epoch 78/300
50/50 [==============================] - 0s 2ms/step - loss: 0.2827 - acc: 0.9400
Epoch 79/300
50/50 [==============================] - 0s 1ms/step - loss: 0.4441 - acc: 0.9200
Epoch 80/300
50/50 [==============================] - 0s 1ms/step - loss: 0.3181 - acc: 0.9200
Epoch 81/300
50/50 [==============================] - 0s 1ms/step - loss: 0.4011 - acc: 0.9200
Epoch 82/300
50/50 [==============================] - 0s 1ms/step - loss: 0.2800 - acc: 0.9200
Epoch 83/300
50/50 [==============================] - 0s 1ms/st

50/50 [==============================] - 0s 1ms/step - loss: 0.2340 - acc: 0.9600
Epoch 159/300
50/50 [==============================] - 0s 1ms/step - loss: 0.2053 - acc: 0.9800
Epoch 160/300
50/50 [==============================] - 0s 1ms/step - loss: 0.2269 - acc: 0.9800
Epoch 161/300
50/50 [==============================] - 0s 1ms/step - loss: 0.3013 - acc: 0.9200
Epoch 162/300
50/50 [==============================] - 0s 2ms/step - loss: 0.2612 - acc: 0.9400
Epoch 163/300
50/50 [==============================] - 0s 2ms/step - loss: 0.3392 - acc: 0.9400
Epoch 164/300
50/50 [==============================] - 0s 2ms/step - loss: 0.2684 - acc: 0.9400
Epoch 165/300
50/50 [==============================] - 0s 2ms/step - loss: 0.1463 - acc: 1.0000
Epoch 166/300
50/50 [==============================] - 1s 11ms/step - loss: 0.1500 - acc: 1.0000
Epoch 167/300
50/50 [==============================] - 0s 2ms/step - loss: 0.2890 - acc: 0.9200
Epoch 168/300
50/50 [==============================] 

50/50 [==============================] - 0s 2ms/step - loss: 0.1526 - acc: 0.9800
Epoch 244/300
50/50 [==============================] - 0s 1ms/step - loss: 0.1976 - acc: 0.9400
Epoch 245/300
50/50 [==============================] - 0s 1ms/step - loss: 0.1817 - acc: 0.9600
Epoch 246/300
50/50 [==============================] - 0s 2ms/step - loss: 0.2656 - acc: 0.9400
Epoch 247/300
50/50 [==============================] - 0s 1ms/step - loss: 0.1882 - acc: 0.9800
Epoch 248/300
50/50 [==============================] - 0s 2ms/step - loss: 0.1254 - acc: 1.0000
Epoch 249/300
50/50 [==============================] - 0s 1ms/step - loss: 0.2140 - acc: 0.9600
Epoch 250/300
50/50 [==============================] - 0s 2ms/step - loss: 0.2342 - acc: 0.9400
Epoch 251/300
50/50 [==============================] - 0s 1ms/step - loss: 0.1649 - acc: 1.0000
Epoch 252/300
50/50 [==============================] - 0s 2ms/step - loss: 0.1253 - acc: 1.0000
Epoch 253/300
50/50 [==============================] -

...

50/50 [==============================] - 0s 2ms/step - loss: 0.0310 - acc: 1.0000
Epoch 25/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0337 - acc: 1.0000
Epoch 26/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0327 - acc: 1.0000
Epoch 27/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0363 - acc: 1.0000
Epoch 28/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0298 - acc: 1.0000
Epoch 29/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0373 - acc: 1.0000
Epoch 30/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0338 - acc: 1.0000
Epoch 31/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0343 - acc: 1.0000
Epoch 32/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0324 - acc: 1.0000
Epoch 33/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0935 - acc: 0.9600
Epoch 34/100
50/50 [==============================] - 0s 2ms/st

50/50 [==============================] - 0s 2ms/step - loss: 0.0310 - acc: 1.0000
Epoch 11/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0294 - acc: 1.0000
Epoch 12/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0327 - acc: 1.0000
Epoch 13/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0375 - acc: 1.0000
Epoch 14/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0323 - acc: 1.0000
Epoch 15/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0299 - acc: 1.0000
Epoch 16/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0274 - acc: 1.0000
Epoch 17/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0342 - acc: 1.0000
Epoch 18/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0396 - acc: 1.0000
Epoch 19/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0913 - acc: 0.9800
Epoch 20/300
50/50 [==============================] - 0s 2ms/st

50/50 [==============================] - 0s 1ms/step - loss: 0.0312 - acc: 1.0000
Epoch 97/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0284 - acc: 1.0000
Epoch 98/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0300 - acc: 1.0000
Epoch 99/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0303 - acc: 1.0000
Epoch 100/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0282 - acc: 1.0000
Epoch 101/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0639 - acc: 0.9800
Epoch 102/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0272 - acc: 1.0000
Epoch 103/300
50/50 [==============================] - 0s 2ms/step - loss: 0.1077 - acc: 0.9600
Epoch 104/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0898 - acc: 1.0000
Epoch 105/300
50/50 [==============================] - 0s 1ms/step - loss: 0.2309 - acc: 0.9800
Epoch 106/300
50/50 [==============================] - 0s

50/50 [==============================] - 0s 2ms/step - loss: 0.0262 - acc: 1.0000
Epoch 182/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0249 - acc: 1.0000
Epoch 183/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0323 - acc: 1.0000
Epoch 184/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0412 - acc: 1.0000
Epoch 185/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0352 - acc: 1.0000
Epoch 186/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0314 - acc: 1.0000
Epoch 187/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0306 - acc: 1.0000
Epoch 188/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0272 - acc: 1.0000
Epoch 189/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0290 - acc: 1.0000
Epoch 190/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0266 - acc: 1.0000
Epoch 191/300
50/50 [==============================] -

50/50 [==============================] - 0s 2ms/step - loss: 0.0742 - acc: 0.9800
Epoch 267/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0296 - acc: 1.0000
Epoch 268/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0285 - acc: 1.0000
Epoch 269/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0818 - acc: 0.9600
Epoch 270/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0497 - acc: 1.0000
Epoch 271/300
50/50 [==============================] - 0s 1ms/step - loss: 0.0311 - acc: 1.0000
Epoch 272/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0617 - acc: 0.9800
Epoch 273/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0343 - acc: 1.0000
Epoch 274/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0883 - acc: 0.9600
Epoch 275/300
50/50 [==============================] - 0s 2ms/step - loss: 0.0434 - acc: 0.9800
Epoch 276/300
50/50 [==============================] -

In [79]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.8982
Score O2 avg: 0.7162


In [ ]:
# Now evaluate with 200 training epochs per trial

In [80]:
model = cnn3(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=100
)
scores_o1, scores_o2 = model_trial(model, 5, nb_epoch=200, burn_period=100)

Trial #0
Epoch 1/100
50/50 [==============================] - 1s 16ms/step - loss: 2.7443 - acc: 0.0600
Epoch 2/100
50/50 [==============================] - 0s 2ms/step - loss: 2.4901 - acc: 0.1600
Epoch 3/100
50/50 [==============================] - 0s 1ms/step - loss: 2.4696 - acc: 0.1200
Epoch 4/100
50/50 [==============================] - 0s 2ms/step - loss: 2.4120 - acc: 0.1400
Epoch 5/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3928 - acc: 0.1000
Epoch 6/100
50/50 [==============================] - 0s 1ms/step - loss: 2.4451 - acc: 0.1000
Epoch 7/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3819 - acc: 0.1400
Epoch 8/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3286 - acc: 0.1000
Epoch 9/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3474 - acc: 0.2000
Epoch 10/100
50/50 [==============================] - 0s 2ms/step - loss: 2.3768 - acc: 0.2000
Epoch 11/100
50/50 [=============================

50/50 [==============================] - 0s 1ms/step - loss: 0.9467 - acc: 0.7600
Epoch 88/100
50/50 [==============================] - 0s 1ms/step - loss: 0.7748 - acc: 0.8400
Epoch 89/100
50/50 [==============================] - 0s 1ms/step - loss: 0.6556 - acc: 0.8400
Epoch 90/100
50/50 [==============================] - 0s 1ms/step - loss: 0.6945 - acc: 0.8200
Epoch 91/100
50/50 [==============================] - 0s 2ms/step - loss: 0.7668 - acc: 0.8200
Epoch 92/100
50/50 [==============================] - 0s 2ms/step - loss: 0.7176 - acc: 0.8200
Epoch 93/100
50/50 [==============================] - 0s 2ms/step - loss: 0.7061 - acc: 0.8200
Epoch 94/100
50/50 [==============================] - 0s 2ms/step - loss: 0.6865 - acc: 0.7800
Epoch 95/100
50/50 [==============================] - 0s 2ms/step - loss: 0.6782 - acc: 0.7800
Epoch 96/100
50/50 [==============================] - 0s 2ms/step - loss: 0.7336 - acc: 0.8000
Epoch 97/100
50/50 [==============================] - 0s 2ms/st

50/50 [==============================] - 0s 2ms/step - loss: 0.3100 - acc: 0.9400
Epoch 74/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2863 - acc: 0.9600
Epoch 75/100
50/50 [==============================] - 0s 2ms/step - loss: 0.3009 - acc: 0.9600
Epoch 76/100
50/50 [==============================] - 0s 2ms/step - loss: 0.3062 - acc: 0.9800
Epoch 77/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2228 - acc: 1.0000
Epoch 78/100
50/50 [==============================] - 0s 2ms/step - loss: 0.3323 - acc: 0.9200
Epoch 79/100
50/50 [==============================] - 0s 1ms/step - loss: 0.2435 - acc: 0.9800
Epoch 80/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2775 - acc: 0.9600
Epoch 81/100
50/50 [==============================] - 0s 2ms/step - loss: 0.4049 - acc: 0.9400
Epoch 82/100
50/50 [==============================] - 0s 2ms/step - loss: 0.4192 - acc: 0.8400
Epoch 83/100
50/50 [==============================] - 0s 2ms/st

Epoch 59/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2069 - acc: 0.9600
Epoch 60/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1796 - acc: 0.9600
Epoch 61/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1558 - acc: 1.0000
Epoch 62/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1671 - acc: 1.0000
Epoch 63/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1919 - acc: 0.9600
Epoch 64/100
50/50 [==============================] - 0s 1ms/step - loss: 0.2031 - acc: 0.9600
Epoch 65/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2420 - acc: 0.9800
Epoch 66/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1955 - acc: 0.9600
Epoch 67/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1904 - acc: 0.9600
Epoch 68/100
50/50 [==============================] - 0s 2ms/step - loss: 0.3295 - acc: 0.8600
Epoch 69/100
50/50 [==============================

50/50 [==============================] - 0s 1ms/step - loss: 0.1992 - acc: 0.9600
Epoch 46/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1271 - acc: 1.0000
Epoch 47/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1367 - acc: 1.0000
Epoch 48/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1785 - acc: 0.9800
Epoch 49/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1575 - acc: 0.9800
Epoch 50/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1064 - acc: 1.0000
Epoch 51/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2745 - acc: 0.9400
Epoch 52/100
50/50 [==============================] - 0s 1ms/step - loss: 0.2334 - acc: 0.9600
Epoch 53/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1372 - acc: 0.9800
Epoch 54/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1541 - acc: 0.9800
Epoch 55/100
50/50 [==============================] - 0s 2ms/st

...

50/50 [==============================] - 0s 2ms/step - loss: 0.0960 - acc: 0.9800
Epoch 3/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0951 - acc: 1.0000
Epoch 4/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1063 - acc: 0.9800
Epoch 5/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2269 - acc: 0.9600
Epoch 6/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1481 - acc: 0.9600
Epoch 7/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0781 - acc: 1.0000
Epoch 8/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0814 - acc: 1.0000
Epoch 9/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0949 - acc: 0.9800
Epoch 10/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1302 - acc: 0.9800
Epoch 11/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1350 - acc: 0.9800
Epoch 12/100
50/50 [==============================] - 0s 2ms/step - lo

50/50 [==============================] - 0s 2ms/step - loss: 0.0740 - acc: 1.0000
Epoch 89/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0693 - acc: 1.0000
Epoch 90/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0738 - acc: 1.0000
Epoch 91/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0764 - acc: 1.0000
Epoch 92/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0791 - acc: 1.0000
Epoch 93/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0692 - acc: 1.0000
Epoch 94/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0827 - acc: 1.0000
Epoch 95/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0901 - acc: 0.9800
Epoch 96/100
50/50 [==============================] - 0s 2ms/step - loss: 0.1114 - acc: 0.9800
Epoch 97/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0749 - acc: 1.0000
Epoch 98/100
50/50 [==============================] - 0s 2ms/st

50/50 [==============================] - 0s 3ms/step - loss: 0.0573 - acc: 1.0000
Epoch 75/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0565 - acc: 1.0000
Epoch 76/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0627 - acc: 1.0000
Epoch 77/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0629 - acc: 1.0000
Epoch 78/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0603 - acc: 1.0000
Epoch 79/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0575 - acc: 1.0000
Epoch 80/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0551 - acc: 1.0000
Epoch 81/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0645 - acc: 1.0000
Epoch 82/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0622 - acc: 1.0000
Epoch 83/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0651 - acc: 1.0000
Epoch 84/100
50/50 [==============================] - 0s 2ms/st

Epoch 60/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0872 - acc: 0.9800
Epoch 61/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0605 - acc: 1.0000
Epoch 62/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0639 - acc: 1.0000
Epoch 63/100
50/50 [==============================] - 0s 1ms/step - loss: 0.2938 - acc: 0.9600
Epoch 64/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0805 - acc: 1.0000
Epoch 65/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0538 - acc: 1.0000
Epoch 66/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0747 - acc: 1.0000
Epoch 67/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1186 - acc: 0.9800
Epoch 68/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0638 - acc: 1.0000
Epoch 69/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0666 - acc: 1.0000
Epoch 70/100
50/50 [==============================

50/50 [==============================] - 0s 1ms/step - loss: 0.1134 - acc: 0.9800
Epoch 47/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0610 - acc: 1.0000
Epoch 48/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0617 - acc: 1.0000
Epoch 49/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0801 - acc: 0.9600
Epoch 50/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0691 - acc: 1.0000
Epoch 51/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0674 - acc: 0.9800
Epoch 52/100
50/50 [==============================] - 0s 1ms/step - loss: 0.1401 - acc: 0.9600
Epoch 53/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0485 - acc: 1.0000
Epoch 54/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0543 - acc: 1.0000
Epoch 55/100
50/50 [==============================] - 0s 1ms/step - loss: 0.0502 - acc: 1.0000
Epoch 56/100
50/50 [==============================] - 0s 1ms/st

In [81]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.8892
Score O2 avg: 0.7440


In [ ]:
# Now evaluate with the standard 100 training epochs per trial

In [83]:
model = cnn3(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(model, 5)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2
)

Trial #0
Epoch 1/20
50/50 [==============================] - 1s 16ms/step - loss: 3.2630 - acc: 0.0200
Epoch 2/20
50/50 [==============================] - 0s 2ms/step - loss: 2.5458 - acc: 0.1200
Epoch 3/20
50/50 [==============================] - 0s 2ms/step - loss: 2.5913 - acc: 0.1400
Epoch 4/20
50/50 [==============================] - 0s 2ms/step - loss: 2.4738 - acc: 0.1200
Epoch 5/20
50/50 [==============================] - 0s 2ms/step - loss: 2.4853 - acc: 0.0400
Epoch 6/20
50/50 [==============================] - 0s 2ms/step - loss: 2.3786 - acc: 0.1800
Epoch 7/20
50/50 [==============================] - 0s 2ms/step - loss: 2.3791 - acc: 0.0800
Epoch 8/20
50/50 [==============================] - 0s 2ms/step - loss: 2.3460 - acc: 0.1600
Epoch 9/20
50/50 [==============================] - 0s 2ms/step - loss: 2.3759 - acc: 0.1800
Epoch 10/20
50/50 [==============================] - 0s 2ms/step - loss: 2.3612 - acc: 0.1000
Epoch 11/20
50/50 [==============================] - 0s 1ms

50/50 [==============================] - 0s 2ms/step - loss: 0.8458 - acc: 0.7800
Epoch 69/80
50/50 [==============================] - 0s 2ms/step - loss: 0.9142 - acc: 0.7000
Epoch 70/80
50/50 [==============================] - 0s 2ms/step - loss: 0.8023 - acc: 0.7600
Epoch 71/80
50/50 [==============================] - 0s 1ms/step - loss: 0.8518 - acc: 0.7400
Epoch 72/80
50/50 [==============================] - 0s 2ms/step - loss: 0.7224 - acc: 0.8200
Epoch 73/80
50/50 [==============================] - 0s 1ms/step - loss: 0.9035 - acc: 0.7400
Epoch 74/80
50/50 [==============================] - 0s 1ms/step - loss: 0.7349 - acc: 0.8000
Epoch 75/80
50/50 [==============================] - 0s 2ms/step - loss: 0.6642 - acc: 0.8200
Epoch 76/80
50/50 [==============================] - 0s 2ms/step - loss: 0.7143 - acc: 0.8200
Epoch 77/80
50/50 [==============================] - 0s 2ms/step - loss: 0.8156 - acc: 0.7600
Epoch 78/80
50/50 [==============================] - 0s 2ms/step - loss:

Epoch 55/80
50/50 [==============================] - 0s 1ms/step - loss: 0.2868 - acc: 0.9400
Epoch 56/80
50/50 [==============================] - 0s 2ms/step - loss: 0.3169 - acc: 0.9200
Epoch 57/80
50/50 [==============================] - 0s 2ms/step - loss: 0.3533 - acc: 0.9400
Epoch 58/80
50/50 [==============================] - 0s 2ms/step - loss: 0.3907 - acc: 0.9200
Epoch 59/80
50/50 [==============================] - 0s 2ms/step - loss: 0.3069 - acc: 0.9600
Epoch 60/80
50/50 [==============================] - 0s 2ms/step - loss: 0.4690 - acc: 0.8400
Epoch 61/80
50/50 [==============================] - 0s 2ms/step - loss: 0.2939 - acc: 0.9400
Epoch 62/80
50/50 [==============================] - 0s 2ms/step - loss: 0.4927 - acc: 0.8600
Epoch 63/80
50/50 [==============================] - 0s 2ms/step - loss: 0.2577 - acc: 0.9800
Epoch 64/80
50/50 [==============================] - 0s 2ms/step - loss: 0.2902 - acc: 0.9000
Epoch 65/80
50/50 [==============================] - 0s 1ms/

50/50 [==============================] - 0s 2ms/step - loss: 0.1850 - acc: 0.9600
Epoch 42/80
50/50 [==============================] - 0s 1ms/step - loss: 0.2307 - acc: 0.9600
Epoch 43/80
50/50 [==============================] - 0s 2ms/step - loss: 0.2343 - acc: 0.9600
Epoch 44/80
50/50 [==============================] - 0s 2ms/step - loss: 0.2146 - acc: 0.9600
Epoch 45/80
50/50 [==============================] - 0s 2ms/step - loss: 0.4919 - acc: 0.8200
Epoch 46/80
50/50 [==============================] - 0s 2ms/step - loss: 0.2332 - acc: 0.9600
Epoch 47/80
50/50 [==============================] - 0s 1ms/step - loss: 0.2825 - acc: 0.9800
Epoch 48/80
50/50 [==============================] - 0s 1ms/step - loss: 0.1869 - acc: 0.9800
Epoch 49/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1913 - acc: 0.9800
Epoch 50/80
50/50 [==============================] - 0s 1ms/step - loss: 0.3301 - acc: 0.9600
Epoch 51/80
50/50 [==============================] - 0s 1ms/step - loss:

Epoch 28/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1796 - acc: 0.9800
Epoch 29/80
50/50 [==============================] - 0s 2ms/step - loss: 0.2868 - acc: 0.9400
Epoch 30/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1366 - acc: 0.9800
Epoch 31/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1381 - acc: 1.0000
Epoch 32/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1349 - acc: 0.9800
Epoch 33/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1417 - acc: 0.9800
Epoch 34/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1715 - acc: 0.9600
Epoch 35/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1530 - acc: 1.0000
Epoch 36/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1963 - acc: 0.9600
Epoch 37/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1596 - acc: 0.9800
Epoch 38/80
50/50 [==============================] - 0s 2ms/

50/50 [==============================] - 0s 2ms/step - loss: 0.1089 - acc: 1.0000
Epoch 15/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1358 - acc: 0.9800
Epoch 16/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1297 - acc: 0.9800
Epoch 17/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1402 - acc: 0.9600
Epoch 18/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1284 - acc: 1.0000
Epoch 19/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1750 - acc: 0.9600
Epoch 20/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1370 - acc: 0.9600
Epoch 21/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1829 - acc: 0.9600
Epoch 22/80
50/50 [==============================] - 0s 2ms/step - loss: 0.1372 - acc: 0.9800
Epoch 23/80
50/50 [==============================] - 0s 2ms/step - loss: 0.0994 - acc: 1.0000
Epoch 24/80
50/50 [==============================] - 0s 2ms/step - loss:

In [84]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.8140
Score O2 avg: 0.7320


In [ ]:
# model 4

In [125]:
model = cnn4(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
50/50 [==============================] - 1s 24ms/step - loss: 2.4828 - acc: 0.1600
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 2.4415 - acc: 0.0800
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 2.3779 - acc: 0.1800
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 2.3564 - acc: 0.1800
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 2.3832 - acc: 0.1200
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 2.3136 - acc: 0.2200
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 2.2765 - acc: 0.2600
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 2.1789 - acc: 0.2600
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 2.1541 - acc: 0.3400
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 2.0035 - acc: 0.3800
Epoch 11/50
50/50 [==============================] - 0s 2ms

50/50 [==============================] - 0s 1ms/step - loss: 0.2145 - acc: 0.9800
Epoch 38/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3211 - acc: 0.9000
Epoch 39/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1999 - acc: 0.9600
Epoch 40/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2760 - acc: 0.9400
Epoch 41/150
50/50 [==============================] - 0s 2ms/step - loss: 0.2116 - acc: 0.9800
Epoch 42/150
50/50 [==============================] - 0s 3ms/step - loss: 0.2127 - acc: 0.9800
Epoch 43/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2941 - acc: 0.9600
Epoch 44/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2337 - acc: 0.9600
Epoch 45/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1795 - acc: 1.0000
Epoch 46/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2850 - acc: 0.9400
Epoch 47/150
50/50 [==============================] - 0s 1ms/st

50/50 [==============================] - 0s 2ms/step - loss: 0.1283 - acc: 0.9800
Epoch 124/150
50/50 [==============================] - 0s 3ms/step - loss: 0.0935 - acc: 1.0000
Epoch 125/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1210 - acc: 0.9800
Epoch 126/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1053 - acc: 1.0000
Epoch 127/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0933 - acc: 1.0000
Epoch 128/150
50/50 [==============================] - 0s 2ms/step - loss: 0.1311 - acc: 0.9800
Epoch 129/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1405 - acc: 0.9800
Epoch 130/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1266 - acc: 0.9800
Epoch 131/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1041 - acc: 1.0000
Epoch 132/150
50/50 [==============================] - 0s 5ms/step - loss: 0.0902 - acc: 1.0000
Epoch 133/150
50/50 [==============================] -

50/50 [==============================] - 0s 1ms/step - loss: 0.1513 - acc: 0.9600
Epoch 10/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0830 - acc: 1.0000
Epoch 11/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0717 - acc: 1.0000
Epoch 12/150
50/50 [==============================] - 0s 2ms/step - loss: 0.1120 - acc: 0.9800
Epoch 13/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0889 - acc: 1.0000
Epoch 14/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0785 - acc: 1.0000
Epoch 15/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0881 - acc: 1.0000
Epoch 16/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0739 - acc: 1.0000
Epoch 17/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0913 - acc: 1.0000
Epoch 18/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0751 - acc: 1.0000
Epoch 19/150
50/50 [==============================] - 0s 1ms/st

50/50 [==============================] - 0s 1ms/step - loss: 0.0569 - acc: 1.0000
Epoch 96/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0609 - acc: 1.0000
Epoch 97/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0969 - acc: 0.9800
Epoch 98/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0629 - acc: 1.0000
Epoch 99/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0543 - acc: 1.0000
Epoch 100/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0514 - acc: 1.0000
Epoch 101/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0552 - acc: 1.0000
Epoch 102/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0613 - acc: 1.0000
Epoch 103/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0547 - acc: 1.0000
Epoch 104/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0599 - acc: 1.0000
Epoch 105/150
50/50 [==============================] - 0s 

...

50/50 [==============================] - 0s 1ms/step - loss: 0.0280 - acc: 1.0000
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.0370 - acc: 1.0000
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.0271 - acc: 1.0000
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.0298 - acc: 1.0000
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.0286 - acc: 1.0000
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.0276 - acc: 1.0000
Epoch 7/50
50/50 [==============================] - 0s 1ms/step - loss: 0.0394 - acc: 1.0000
Epoch 8/50
50/50 [==============================] - 0s 1ms/step - loss: 0.0275 - acc: 1.0000
Epoch 9/50
50/50 [==============================] - 0s 1ms/step - loss: 0.0269 - acc: 1.0000
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.0262 - acc: 1.0000
Epoch 11/50
50/50 [==============================] - 0s 1ms/step - loss: 0.0283 

50/50 [==============================] - 0s 1ms/step - loss: 0.0243 - acc: 1.0000
Epoch 39/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0262 - acc: 1.0000
Epoch 40/150
50/50 [==============================] - 0s 5ms/step - loss: 0.0230 - acc: 1.0000
Epoch 41/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0247 - acc: 1.0000
Epoch 42/150
50/50 [==============================] - 0s 5ms/step - loss: 0.0227 - acc: 1.0000
Epoch 43/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0234 - acc: 1.0000
Epoch 44/150
50/50 [==============================] - 0s 5ms/step - loss: 0.0227 - acc: 1.0000
Epoch 45/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0272 - acc: 1.0000
Epoch 46/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0234 - acc: 1.0000
Epoch 47/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0220 - acc: 1.0000
Epoch 48/150
50/50 [==============================] - 0s 2ms/st

50/50 [==============================] - 0s 4ms/step - loss: 0.0185 - acc: 1.0000
Epoch 125/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0655 - acc: 0.9800
Epoch 126/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0220 - acc: 1.0000
Epoch 127/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0206 - acc: 1.0000
Epoch 128/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0247 - acc: 1.0000
Epoch 129/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0555 - acc: 0.9800
Epoch 130/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0278 - acc: 1.0000
Epoch 131/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0234 - acc: 1.0000
Epoch 132/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0194 - acc: 1.0000
Epoch 133/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0219 - acc: 1.0000
Epoch 134/150
50/50 [==============================] -

50/50 [==============================] - 0s 2ms/step - loss: 0.0160 - acc: 1.0000
Epoch 11/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0173 - acc: 1.0000
Epoch 12/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0165 - acc: 1.0000
Epoch 13/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0163 - acc: 1.0000
Epoch 14/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0241 - acc: 1.0000
Epoch 15/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0188 - acc: 1.0000
Epoch 16/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0179 - acc: 1.0000
Epoch 17/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0163 - acc: 1.0000
Epoch 18/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0228 - acc: 1.0000
Epoch 19/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0282 - acc: 1.0000
Epoch 20/150
50/50 [==============================] - 0s 1ms/st

50/50 [==============================] - 0s 1ms/step - loss: 0.0153 - acc: 1.0000
Epoch 97/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0292 - acc: 1.0000
Epoch 98/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0168 - acc: 1.0000
Epoch 99/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0142 - acc: 1.0000
Epoch 100/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0144 - acc: 1.0000
Epoch 101/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0156 - acc: 1.0000
Epoch 102/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0148 - acc: 1.0000
Epoch 103/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0143 - acc: 1.0000
Epoch 104/150
50/50 [==============================] - 0s 2ms/step - loss: 0.0138 - acc: 1.0000
Epoch 105/150
50/50 [==============================] - 0s 1ms/step - loss: 0.0168 - acc: 1.0000
Epoch 106/150
50/50 [==============================] - 0s

In [126]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.8556
Score O2 avg: 0.7506


In [ ]:
# model 5

In [121]:
model = cnn5(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
50/50 [==============================] - 1s 21ms/step - loss: 2.6548 - acc: 0.0400
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 2.3342 - acc: 0.1600
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 2.2734 - acc: 0.1800
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 2.3520 - acc: 0.1600
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 2.2752 - acc: 0.2200
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 2.2421 - acc: 0.1000
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 2.1443 - acc: 0.2400
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 2.1303 - acc: 0.2600
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 2.1846 - acc: 0.2200
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 2.2035 - acc: 0.1800
Epoch 11/50
50/50 [==============================] - 0s 2ms

50/50 [==============================] - 0s 2ms/step - loss: 0.8986 - acc: 0.6400
Epoch 38/150
50/50 [==============================] - 0s 2ms/step - loss: 0.8605 - acc: 0.7000
Epoch 39/150
50/50 [==============================] - 0s 2ms/step - loss: 0.9682 - acc: 0.6800
Epoch 40/150
50/50 [==============================] - 0s 2ms/step - loss: 0.7885 - acc: 0.7400
Epoch 41/150
50/50 [==============================] - 0s 2ms/step - loss: 0.8431 - acc: 0.7600
Epoch 42/150
50/50 [==============================] - 0s 2ms/step - loss: 0.7970 - acc: 0.7400
Epoch 43/150
50/50 [==============================] - 0s 2ms/step - loss: 0.5278 - acc: 0.8400
Epoch 44/150
50/50 [==============================] - 0s 1ms/step - loss: 0.9865 - acc: 0.5800
Epoch 45/150
50/50 [==============================] - 0s 2ms/step - loss: 0.6798 - acc: 0.7400
Epoch 46/150
50/50 [==============================] - 0s 2ms/step - loss: 0.8700 - acc: 0.6600
Epoch 47/150
50/50 [==============================] - 0s 2ms/st

50/50 [==============================] - 0s 2ms/step - loss: 0.5418 - acc: 0.7600
Epoch 124/150
50/50 [==============================] - 0s 2ms/step - loss: 0.5401 - acc: 0.8200
Epoch 125/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3612 - acc: 0.8800
Epoch 126/150
50/50 [==============================] - 0s 1ms/step - loss: 0.6532 - acc: 0.7800
Epoch 127/150
50/50 [==============================] - 0s 1ms/step - loss: 0.4038 - acc: 0.8600
Epoch 128/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4928 - acc: 0.8200
Epoch 129/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4817 - acc: 0.8400
Epoch 130/150
50/50 [==============================] - 0s 1ms/step - loss: 0.4893 - acc: 0.8200
Epoch 131/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4205 - acc: 0.9000
Epoch 132/150
50/50 [==============================] - 0s 1ms/step - loss: 0.5412 - acc: 0.8200
Epoch 133/150
50/50 [==============================] -

50/50 [==============================] - 0s 2ms/step - loss: 0.4298 - acc: 0.8800
Epoch 10/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3636 - acc: 0.8600
Epoch 11/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2586 - acc: 0.9800
Epoch 12/150
50/50 [==============================] - 0s 1ms/step - loss: 0.5008 - acc: 0.8600
Epoch 13/150
50/50 [==============================] - 0s 2ms/step - loss: 0.2812 - acc: 0.9400
Epoch 14/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4624 - acc: 0.8400
Epoch 15/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2848 - acc: 0.9400
Epoch 16/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3633 - acc: 0.8200
Epoch 17/150
50/50 [==============================] - 0s 1ms/step - loss: 0.4865 - acc: 0.8400
Epoch 18/150
50/50 [==============================] - 0s 1ms/step - loss: 0.4204 - acc: 0.8600
Epoch 19/150
50/50 [==============================] - 0s 2ms/st

50/50 [==============================] - 0s 1ms/step - loss: 0.3162 - acc: 0.9200
Epoch 96/150
50/50 [==============================] - 0s 1ms/step - loss: 0.5676 - acc: 0.7600
Epoch 97/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3804 - acc: 0.8600
Epoch 98/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2697 - acc: 0.9400
Epoch 99/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2016 - acc: 0.9400
Epoch 100/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3330 - acc: 0.8800
Epoch 101/150
50/50 [==============================] - 0s 1ms/step - loss: 0.4133 - acc: 0.8600
Epoch 102/150
50/50 [==============================] - 0s 1ms/step - loss: 0.4779 - acc: 0.8000
Epoch 103/150
50/50 [==============================] - 0s 1ms/step - loss: 0.4834 - acc: 0.8800
Epoch 104/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3957 - acc: 0.9000
Epoch 105/150
50/50 [==============================] - 0s 

...

50/50 [==============================] - 0s 1ms/step - loss: 0.3528 - acc: 0.8600
Epoch 2/50
50/50 [==============================] - 0s 1ms/step - loss: 0.1573 - acc: 0.9800
Epoch 3/50
50/50 [==============================] - 0s 1ms/step - loss: 0.3421 - acc: 0.9000
Epoch 4/50
50/50 [==============================] - 0s 1ms/step - loss: 0.2040 - acc: 0.9600
Epoch 5/50
50/50 [==============================] - 0s 1ms/step - loss: 0.3287 - acc: 0.8800
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.1641 - acc: 0.9800
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2954 - acc: 0.9200
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2065 - acc: 0.9400
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2244 - acc: 0.9400
Epoch 10/50
50/50 [==============================] - 0s 1ms/step - loss: 0.1804 - acc: 0.9600
Epoch 11/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2861 

50/50 [==============================] - 0s 1ms/step - loss: 0.2658 - acc: 0.9200
Epoch 39/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1736 - acc: 0.9200
Epoch 40/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3393 - acc: 0.9200
Epoch 41/150
50/50 [==============================] - 0s 2ms/step - loss: 0.4349 - acc: 0.8400
Epoch 42/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2311 - acc: 0.9400
Epoch 43/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3123 - acc: 0.9000
Epoch 44/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1988 - acc: 0.9600
Epoch 45/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2631 - acc: 0.9000
Epoch 46/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2963 - acc: 0.9000
Epoch 47/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2179 - acc: 0.9400
Epoch 48/150
50/50 [==============================] - 0s 1ms/st

50/50 [==============================] - 0s 2ms/step - loss: 0.1862 - acc: 0.9200
Epoch 125/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1002 - acc: 0.9800
Epoch 126/150
50/50 [==============================] - 0s 2ms/step - loss: 0.1974 - acc: 0.9600
Epoch 127/150
50/50 [==============================] - 0s 2ms/step - loss: 0.2996 - acc: 0.8400
Epoch 128/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1325 - acc: 0.9800
Epoch 129/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3257 - acc: 0.9000
Epoch 130/150
50/50 [==============================] - 0s 1ms/step - loss: 0.4802 - acc: 0.8200
Epoch 131/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1301 - acc: 0.9600
Epoch 132/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2425 - acc: 0.9200
Epoch 133/150
50/50 [==============================] - 0s 1ms/step - loss: 0.3636 - acc: 0.9200
Epoch 134/150
50/50 [==============================] -

50/50 [==============================] - 0s 1ms/step - loss: 0.2967 - acc: 0.9000
Epoch 11/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1985 - acc: 0.9400
Epoch 12/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1886 - acc: 0.9200
Epoch 13/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2015 - acc: 0.9400
Epoch 14/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2124 - acc: 0.9600
Epoch 15/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1988 - acc: 0.9400
Epoch 16/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2421 - acc: 0.9200
Epoch 17/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2605 - acc: 0.9200
Epoch 18/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1821 - acc: 0.9400
Epoch 19/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1408 - acc: 0.9800
Epoch 20/150
50/50 [==============================] - 0s 1ms/st

50/50 [==============================] - 0s 1ms/step - loss: 0.2871 - acc: 0.8800
Epoch 97/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2221 - acc: 0.9400
Epoch 98/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1295 - acc: 0.9800
Epoch 99/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1749 - acc: 0.9600
Epoch 100/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2663 - acc: 0.8800
Epoch 101/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2373 - acc: 0.9000
Epoch 102/150
50/50 [==============================] - 0s 2ms/step - loss: 0.2574 - acc: 0.9000
Epoch 103/150
50/50 [==============================] - 0s 1ms/step - loss: 0.1729 - acc: 0.9600
Epoch 104/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2436 - acc: 0.9200
Epoch 105/150
50/50 [==============================] - 0s 1ms/step - loss: 0.2330 - acc: 0.9200
Epoch 106/150
50/50 [==============================] - 0s

In [122]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.8792
Score O2 avg: 0.7100


# Let's try 4&5 it with other parameter sets...

In [127]:
nb_categories = 20
nb_exemplars = 5
X_train, Y_train, X_test_o1, X_test_o2 = get_data(nb_categories, nb_exemplars)

In [97]:
model = cnn4(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
100/100 [==============================] - 1s 10ms/step - loss: 3.1942 - acc: 0.0400
Epoch 2/50
100/100 [==============================] - 0s 2ms/step - loss: 3.0914 - acc: 0.0500
Epoch 3/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0579 - acc: 0.0800
Epoch 4/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0009 - acc: 0.0900
Epoch 5/50
100/100 [==============================] - 0s 1ms/step - loss: 2.9801 - acc: 0.1000
Epoch 6/50
100/100 [==============================] - 0s 1ms/step - loss: 2.9522 - acc: 0.1400
Epoch 7/50
100/100 [==============================] - 0s 1ms/step - loss: 2.9365 - acc: 0.1700
Epoch 8/50
100/100 [==============================] - 0s 1ms/step - loss: 2.8443 - acc: 0.2100
Epoch 9/50
100/100 [==============================] - 0s 1ms/step - loss: 2.8412 - acc: 0.1700
Epoch 10/50
100/100 [==============================] - 0s 1ms/step - loss: 2.7668 - acc: 0.1900
Epoch 11/50
100/100 [==================

100/100 [==============================] - 0s 1ms/step - loss: 0.4328 - acc: 0.9300
Epoch 37/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3339 - acc: 0.9200
Epoch 38/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3159 - acc: 0.9400
Epoch 39/150
100/100 [==============================] - 0s 1ms/step - loss: 0.2585 - acc: 0.9800
Epoch 40/150
100/100 [==============================] - 0s 2ms/step - loss: 0.2746 - acc: 0.9700
Epoch 41/150
100/100 [==============================] - 0s 1ms/step - loss: 0.2992 - acc: 0.9700
Epoch 42/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3995 - acc: 0.9000
Epoch 43/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4560 - acc: 0.8900
Epoch 44/150
100/100 [==============================] - 0s 2ms/step - loss: 0.2714 - acc: 0.9600
Epoch 45/150
100/100 [==============================] - 0s 1ms/step - loss: 0.2870 - acc: 0.9500
Epoch 46/150
100/100 [=====================

100/100 [==============================] - 0s 1ms/step - loss: 0.1580 - acc: 0.9800
Epoch 121/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1185 - acc: 1.0000
Epoch 122/150
100/100 [==============================] - 0s 2ms/step - loss: 0.1147 - acc: 1.0000
Epoch 123/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1612 - acc: 0.9800
Epoch 124/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1311 - acc: 0.9900
Epoch 125/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1335 - acc: 0.9900
Epoch 126/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1495 - acc: 0.9900
Epoch 127/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1332 - acc: 0.9900
Epoch 128/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1358 - acc: 1.0000
Epoch 129/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1359 - acc: 0.9800
Epoch 130/150
100/100 [===========

100/100 [==============================] - 0s 1ms/step - loss: 0.1178 - acc: 0.9700
Epoch 5/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1027 - acc: 1.0000
Epoch 6/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0938 - acc: 1.0000
Epoch 7/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0923 - acc: 1.0000
Epoch 8/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0816 - acc: 1.0000
Epoch 9/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1023 - acc: 0.9900
Epoch 10/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0960 - acc: 1.0000
Epoch 11/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0962 - acc: 0.9900
Epoch 12/150
100/100 [==============================] - 0s 1ms/step - loss: 0.1067 - acc: 0.9900
Epoch 13/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0975 - acc: 1.0000
Epoch 14/150
100/100 [==========================

100/100 [==============================] - 0s 1ms/step - loss: 0.0617 - acc: 1.0000
Epoch 89/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0701 - acc: 1.0000
Epoch 90/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0647 - acc: 1.0000
Epoch 91/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0936 - acc: 0.9800
Epoch 92/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0643 - acc: 1.0000
Epoch 93/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0619 - acc: 1.0000
Epoch 94/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0638 - acc: 1.0000
Epoch 95/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0756 - acc: 0.9900
Epoch 96/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0731 - acc: 1.0000
Epoch 97/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0607 - acc: 1.0000
Epoch 98/150
100/100 [=====================

...

100/100 [==============================] - 0s 1ms/step - loss: 0.0338 - acc: 1.0000
Epoch 141/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0297 - acc: 1.0000
Epoch 142/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0297 - acc: 1.0000
Epoch 143/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0344 - acc: 1.0000
Epoch 144/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0324 - acc: 1.0000
Epoch 145/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0425 - acc: 0.9900
Epoch 146/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0398 - acc: 1.0000
Epoch 147/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0451 - acc: 0.9900
Epoch 148/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0454 - acc: 1.0000
Epoch 149/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0332 - acc: 1.0000
Epoch 150/150
100/100 [===========

100/100 [==============================] - 0s 1ms/step - loss: 0.0297 - acc: 1.0000
Epoch 25/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0270 - acc: 1.0000
Epoch 26/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0298 - acc: 1.0000
Epoch 27/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0306 - acc: 1.0000
Epoch 28/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0329 - acc: 1.0000
Epoch 29/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0282 - acc: 1.0000
Epoch 30/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0347 - acc: 1.0000
Epoch 31/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0249 - acc: 1.0000
Epoch 32/150
100/100 [==============================] - 0s 3ms/step - loss: 0.0261 - acc: 1.0000
Epoch 33/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0263 - acc: 1.0000
Epoch 34/150
100/100 [=====================

100/100 [==============================] - 0s 1ms/step - loss: 0.0277 - acc: 1.0000
Epoch 109/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0273 - acc: 1.0000
Epoch 110/150
100/100 [==============================] - 0s 2ms/step - loss: 0.0209 - acc: 1.0000
Epoch 111/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0196 - acc: 1.0000
Epoch 112/150
100/100 [==============================] - 0s 2ms/step - loss: 0.0205 - acc: 1.0000
Epoch 113/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0669 - acc: 0.9700
Epoch 114/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0227 - acc: 1.0000
Epoch 115/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0235 - acc: 1.0000
Epoch 116/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0292 - acc: 0.9900
Epoch 117/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0299 - acc: 1.0000
Epoch 118/150
100/100 [===========

100/100 [==============================] - 0s 1ms/step - loss: 0.0206 - acc: 1.0000
Epoch 43/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0217 - acc: 1.0000
Epoch 44/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0266 - acc: 1.0000
Epoch 45/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0262 - acc: 1.0000
Epoch 46/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0191 - acc: 1.0000
Epoch 47/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0176 - acc: 1.0000
Epoch 48/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0187 - acc: 1.0000
Epoch 49/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0277 - acc: 1.0000
Epoch 50/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0210 - acc: 1.0000
Epoch 1/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0169 - acc: 1.0000
Epoch 2/150
100/100 [==============================]

100/100 [==============================] - 0s 2ms/step - loss: 0.0156 - acc: 1.0000
Epoch 77/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0189 - acc: 1.0000
Epoch 78/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0716 - acc: 0.9900
Epoch 79/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0218 - acc: 1.0000
Epoch 80/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0219 - acc: 1.0000
Epoch 81/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0182 - acc: 1.0000
Epoch 82/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0179 - acc: 1.0000
Epoch 83/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0184 - acc: 1.0000
Epoch 84/150
100/100 [==============================] - 0s 2ms/step - loss: 0.0157 - acc: 1.0000
Epoch 85/150
100/100 [==============================] - 0s 1ms/step - loss: 0.0152 - acc: 1.0000
Epoch 86/150
100/100 [=====================

In [98]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.9162
Score O2 avg: 0.8606


In [128]:
model = cnn5(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
100/100 [==============================] - 1s 13ms/step - loss: 3.1938 - acc: 0.0500
Epoch 2/50
100/100 [==============================] - 0s 1ms/step - loss: 3.1174 - acc: 0.0500
Epoch 3/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0946 - acc: 0.0500
Epoch 4/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0668 - acc: 0.0500
Epoch 5/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0745 - acc: 0.0700
Epoch 6/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0591 - acc: 0.1200
Epoch 7/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0661 - acc: 0.0700
Epoch 8/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0525 - acc: 0.0800
Epoch 9/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0356 - acc: 0.1300
Epoch 10/50
100/100 [==============================] - 0s 1ms/step - loss: 3.0321 - acc: 0.1100
Epoch 11/50
100/100 [==================

100/100 [==============================] - 0s 1ms/step - loss: 1.8913 - acc: 0.4000
Epoch 37/150
100/100 [==============================] - 0s 1ms/step - loss: 1.6052 - acc: 0.4800
Epoch 38/150
100/100 [==============================] - 0s 2ms/step - loss: 1.5795 - acc: 0.5200
Epoch 39/150
100/100 [==============================] - 0s 1ms/step - loss: 1.7083 - acc: 0.4700
Epoch 40/150
100/100 [==============================] - 0s 1ms/step - loss: 1.7301 - acc: 0.4400
Epoch 41/150
100/100 [==============================] - 0s 1ms/step - loss: 1.6910 - acc: 0.4600
Epoch 42/150
100/100 [==============================] - 0s 1ms/step - loss: 1.6545 - acc: 0.4500
Epoch 43/150
100/100 [==============================] - 0s 2ms/step - loss: 1.7296 - acc: 0.5100
Epoch 44/150
100/100 [==============================] - 0s 1ms/step - loss: 1.5818 - acc: 0.4700
Epoch 45/150
100/100 [==============================] - 0s 1ms/step - loss: 1.4864 - acc: 0.5500
Epoch 46/150
100/100 [=====================

100/100 [==============================] - 0s 1ms/step - loss: 1.0901 - acc: 0.6400
Epoch 121/150
100/100 [==============================] - 0s 2ms/step - loss: 1.1953 - acc: 0.5800
Epoch 122/150
100/100 [==============================] - 0s 2ms/step - loss: 0.7737 - acc: 0.7400
Epoch 123/150
100/100 [==============================] - 0s 1ms/step - loss: 0.9901 - acc: 0.6300
Epoch 124/150
100/100 [==============================] - 0s 1ms/step - loss: 1.0774 - acc: 0.6400
Epoch 125/150
100/100 [==============================] - 0s 1ms/step - loss: 1.2295 - acc: 0.6300
Epoch 126/150
100/100 [==============================] - 0s 1ms/step - loss: 1.3401 - acc: 0.6500
Epoch 127/150
100/100 [==============================] - 0s 1ms/step - loss: 1.1142 - acc: 0.6700
Epoch 128/150
100/100 [==============================] - 0s 1ms/step - loss: 0.9315 - acc: 0.7300
Epoch 129/150
100/100 [==============================] - 0s 1ms/step - loss: 0.8973 - acc: 0.7400
Epoch 130/150
100/100 [===========

100/100 [==============================] - 0s 2ms/step - loss: 0.7344 - acc: 0.7400
Epoch 5/150
100/100 [==============================] - 0s 1ms/step - loss: 0.9460 - acc: 0.6700
Epoch 6/150
100/100 [==============================] - 0s 1ms/step - loss: 1.0271 - acc: 0.6700
Epoch 7/150
100/100 [==============================] - 0s 1ms/step - loss: 0.9510 - acc: 0.7000
Epoch 8/150
100/100 [==============================] - 0s 1ms/step - loss: 0.8868 - acc: 0.7000
Epoch 9/150
100/100 [==============================] - 0s 1ms/step - loss: 0.8074 - acc: 0.7700
Epoch 10/150
100/100 [==============================] - 0s 2ms/step - loss: 0.7308 - acc: 0.7900
Epoch 11/150
100/100 [==============================] - 0s 1ms/step - loss: 0.8610 - acc: 0.7100
Epoch 12/150
100/100 [==============================] - 0s 1ms/step - loss: 0.7847 - acc: 0.7700
Epoch 13/150
100/100 [==============================] - 0s 1ms/step - loss: 0.8585 - acc: 0.7200
Epoch 14/150
100/100 [==========================

100/100 [==============================] - 0s 1ms/step - loss: 0.7712 - acc: 0.8300
Epoch 89/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6815 - acc: 0.7700
Epoch 90/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6978 - acc: 0.8000
Epoch 91/150
100/100 [==============================] - 0s 1ms/step - loss: 0.7445 - acc: 0.7700
Epoch 92/150
100/100 [==============================] - 0s 1ms/step - loss: 0.7587 - acc: 0.7400
Epoch 93/150
100/100 [==============================] - 0s 1ms/step - loss: 0.8462 - acc: 0.7200
Epoch 94/150
100/100 [==============================] - 0s 1ms/step - loss: 0.7584 - acc: 0.7600
Epoch 95/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6970 - acc: 0.8100
Epoch 96/150
100/100 [==============================] - 0s 1ms/step - loss: 0.9813 - acc: 0.7000
Epoch 97/150
100/100 [==============================] - 0s 1ms/step - loss: 0.7756 - acc: 0.7500
Epoch 98/150
100/100 [=====================

...

100/100 [==============================] - 0s 1ms/step - loss: 0.7042 - acc: 0.7800
Epoch 141/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4773 - acc: 0.8800
Epoch 142/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5221 - acc: 0.8200
Epoch 143/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6866 - acc: 0.7800
Epoch 144/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6430 - acc: 0.7800
Epoch 145/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6430 - acc: 0.8000
Epoch 146/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6104 - acc: 0.7800
Epoch 147/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6903 - acc: 0.7700
Epoch 148/150
100/100 [==============================] - 0s 1ms/step - loss: 0.7880 - acc: 0.7500
Epoch 149/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5224 - acc: 0.8200
Epoch 150/150
100/100 [===========

100/100 [==============================] - 0s 1ms/step - loss: 0.5002 - acc: 0.8300
Epoch 25/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5658 - acc: 0.7900
Epoch 26/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4297 - acc: 0.8700
Epoch 27/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5466 - acc: 0.7500
Epoch 28/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4559 - acc: 0.8800
Epoch 29/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6156 - acc: 0.7800
Epoch 30/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5293 - acc: 0.8300
Epoch 31/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5645 - acc: 0.8100
Epoch 32/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4762 - acc: 0.8200
Epoch 33/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5164 - acc: 0.8400
Epoch 34/150
100/100 [=====================

100/100 [==============================] - 0s 1ms/step - loss: 0.4524 - acc: 0.8600
Epoch 109/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4247 - acc: 0.8400
Epoch 110/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3864 - acc: 0.9000
Epoch 111/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4289 - acc: 0.8500
Epoch 112/150
100/100 [==============================] - 0s 1ms/step - loss: 0.3959 - acc: 0.8700
Epoch 113/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4584 - acc: 0.8400
Epoch 114/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6196 - acc: 0.7800
Epoch 115/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4782 - acc: 0.8700
Epoch 116/150
100/100 [==============================] - 0s 1ms/step - loss: 0.6134 - acc: 0.8000
Epoch 117/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5662 - acc: 0.7800
Epoch 118/150
100/100 [===========

100/100 [==============================] - 0s 1ms/step - loss: 0.4920 - acc: 0.8500
Epoch 43/50
100/100 [==============================] - 0s 1ms/step - loss: 0.6255 - acc: 0.7900
Epoch 44/50
100/100 [==============================] - 0s 1ms/step - loss: 0.4746 - acc: 0.8800
Epoch 45/50
100/100 [==============================] - 0s 1ms/step - loss: 0.4613 - acc: 0.8700
Epoch 46/50
100/100 [==============================] - 0s 1ms/step - loss: 0.7515 - acc: 0.7900
Epoch 47/50
100/100 [==============================] - 0s 1ms/step - loss: 0.5647 - acc: 0.8200
Epoch 48/50
100/100 [==============================] - 0s 1ms/step - loss: 0.5262 - acc: 0.8700
Epoch 49/50
100/100 [==============================] - 0s 1ms/step - loss: 0.5724 - acc: 0.8000
Epoch 50/50
100/100 [==============================] - 0s 1ms/step - loss: 0.5619 - acc: 0.8100
Epoch 1/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5922 - acc: 0.8000
Epoch 2/150
100/100 [==============================]

100/100 [==============================] - 0s 1ms/step - loss: 0.4067 - acc: 0.8600
Epoch 77/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4671 - acc: 0.8200
Epoch 78/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4293 - acc: 0.8600
Epoch 79/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4462 - acc: 0.8400
Epoch 80/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4092 - acc: 0.9000
Epoch 81/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5567 - acc: 0.8300
Epoch 82/150
100/100 [==============================] - 0s 1ms/step - loss: 0.4945 - acc: 0.8300
Epoch 83/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5778 - acc: 0.8100
Epoch 84/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5189 - acc: 0.8400
Epoch 85/150
100/100 [==============================] - 0s 1ms/step - loss: 0.5188 - acc: 0.8300
Epoch 86/150
100/100 [=====================

In [129]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.8764
Score O2 avg: 0.8250


In [130]:
nb_categories = 5
nb_exemplars = 3
X_train, Y_train, X_test_o1, X_test_o2 = get_data(nb_categories, nb_exemplars)

In [131]:
model = cnn4(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
15/15 [==============================] - 2s 117ms/step - loss: 1.6929 - acc: 0.2667
Epoch 2/50
15/15 [==============================] - 0s 2ms/step - loss: 1.7620 - acc: 0.2000
Epoch 3/50
15/15 [==============================] - 0s 2ms/step - loss: 1.7041 - acc: 0.4667
Epoch 4/50
15/15 [==============================] - 0s 2ms/step - loss: 1.6563 - acc: 0.2667
Epoch 5/50
15/15 [==============================] - 0s 2ms/step - loss: 1.6717 - acc: 0.3333
Epoch 6/50
15/15 [==============================] - 0s 2ms/step - loss: 1.6762 - acc: 0.3333
Epoch 7/50
15/15 [==============================] - 0s 2ms/step - loss: 1.6341 - acc: 0.3333
Epoch 8/50
15/15 [==============================] - 0s 2ms/step - loss: 1.6366 - acc: 0.2667
Epoch 9/50
15/15 [==============================] - 0s 2ms/step - loss: 1.7096 - acc: 0.2000
Epoch 10/50
15/15 [==============================] - 0s 2ms/step - loss: 1.6417 - acc: 0.2000
Epoch 11/50
15/15 [==============================] - 0s 2m

Epoch 38/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3097 - acc: 0.9333
Epoch 39/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2554 - acc: 0.9333
Epoch 40/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3551 - acc: 0.9333
Epoch 41/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3533 - acc: 0.8667
Epoch 42/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2708 - acc: 1.0000
Epoch 43/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1892 - acc: 1.0000
Epoch 44/150
15/15 [==============================] - 0s 1ms/step - loss: 0.4263 - acc: 0.8667
Epoch 45/150
15/15 [==============================] - 0s 2ms/step - loss: 0.4195 - acc: 0.9333
Epoch 46/150
15/15 [==============================] - 0s 2ms/step - loss: 0.2690 - acc: 0.8667
Epoch 47/150
15/15 [==============================] - 0s 2ms/step - loss: 0.3949 - acc: 0.8667
Epoch 48/150
15/15 [==============================

Epoch 124/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1357 - acc: 1.0000
Epoch 125/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1270 - acc: 1.0000
Epoch 126/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1303 - acc: 1.0000
Epoch 127/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1297 - acc: 1.0000
Epoch 128/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1146 - acc: 1.0000
Epoch 129/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1132 - acc: 1.0000
Epoch 130/150
15/15 [==============================] - 0s 2ms/step - loss: 0.2839 - acc: 0.9333
Epoch 131/150
15/15 [==============================] - 0s 2ms/step - loss: 0.2050 - acc: 0.9333
Epoch 132/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1467 - acc: 1.0000
Epoch 133/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1938 - acc: 0.9333
Epoch 134/150
15/15 [===================

Epoch 11/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1030 - acc: 1.0000
Epoch 12/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1021 - acc: 1.0000
Epoch 13/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0987 - acc: 1.0000
Epoch 14/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1016 - acc: 1.0000
Epoch 15/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0978 - acc: 1.0000
Epoch 16/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1019 - acc: 1.0000
Epoch 17/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1069 - acc: 1.0000
Epoch 18/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1004 - acc: 1.0000
Epoch 19/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0964 - acc: 1.0000
Epoch 20/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1810 - acc: 0.9333
Epoch 21/150
15/15 [==============================

Epoch 98/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0942 - acc: 1.0000
Epoch 99/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0857 - acc: 1.0000
Epoch 100/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1011 - acc: 1.0000
Epoch 101/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0885 - acc: 1.0000
Epoch 102/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1061 - acc: 1.0000
Epoch 103/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0905 - acc: 1.0000
Epoch 104/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0861 - acc: 1.0000
Epoch 105/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0846 - acc: 1.0000
Epoch 106/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0876 - acc: 1.0000
Epoch 107/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0844 - acc: 1.0000
Epoch 108/150
15/15 [=====================

...

Epoch 7/50
15/15 [==============================] - 0s 1ms/step - loss: 0.0459 - acc: 1.0000
Epoch 8/50
15/15 [==============================] - 0s 1ms/step - loss: 0.0455 - acc: 1.0000
Epoch 9/50
15/15 [==============================] - 0s 1ms/step - loss: 0.0450 - acc: 1.0000
Epoch 10/50
15/15 [==============================] - 0s 1ms/step - loss: 0.0448 - acc: 1.0000
Epoch 11/50
15/15 [==============================] - 0s 1ms/step - loss: 0.0450 - acc: 1.0000
Epoch 12/50
15/15 [==============================] - 0s 2ms/step - loss: 0.0444 - acc: 1.0000
Epoch 13/50
15/15 [==============================] - 0s 1ms/step - loss: 0.0440 - acc: 1.0000
Epoch 14/50
15/15 [==============================] - 0s 1ms/step - loss: 0.0451 - acc: 1.0000
Epoch 15/50
15/15 [==============================] - 0s 1ms/step - loss: 0.0440 - acc: 1.0000
Epoch 16/50
15/15 [==============================] - 0s 2ms/step - loss: 0.0433 - acc: 1.0000
Epoch 17/50
15/15 [==============================] - 0s 1ms/ste

Epoch 44/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0379 - acc: 1.0000
Epoch 45/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0366 - acc: 1.0000
Epoch 46/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0353 - acc: 1.0000
Epoch 47/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0410 - acc: 1.0000
Epoch 48/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0365 - acc: 1.0000
Epoch 49/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0368 - acc: 1.0000
Epoch 50/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0365 - acc: 1.0000
Epoch 51/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0354 - acc: 1.0000
Epoch 52/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0589 - acc: 1.0000
Epoch 53/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0357 - acc: 1.0000
Epoch 54/150
15/15 [==============================

Epoch 130/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0274 - acc: 1.0000
Epoch 131/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0263 - acc: 1.0000
Epoch 132/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0268 - acc: 1.0000
Epoch 133/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0273 - acc: 1.0000
Epoch 134/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0310 - acc: 1.0000
Epoch 135/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0341 - acc: 1.0000
Epoch 136/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0294 - acc: 1.0000
Epoch 137/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0262 - acc: 1.0000
Epoch 138/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1896 - acc: 0.9333
Epoch 139/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0492 - acc: 1.0000
Epoch 140/150
15/15 [===================

Epoch 17/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0223 - acc: 1.0000
Epoch 18/150
15/15 [==============================] - 0s 3ms/step - loss: 0.0222 - acc: 1.0000
Epoch 19/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0248 - acc: 1.0000
Epoch 20/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0251 - acc: 1.0000
Epoch 21/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0226 - acc: 1.0000
Epoch 22/150
15/15 [==============================] - 0s 3ms/step - loss: 0.0221 - acc: 1.0000
Epoch 23/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0229 - acc: 1.0000
Epoch 24/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0230 - acc: 1.0000
Epoch 25/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0223 - acc: 1.0000
Epoch 26/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0218 - acc: 1.0000
Epoch 27/150
15/15 [==============================

Epoch 104/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0193 - acc: 1.0000
Epoch 105/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0186 - acc: 1.0000
Epoch 106/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0189 - acc: 1.0000
Epoch 107/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0223 - acc: 1.0000
Epoch 108/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0187 - acc: 1.0000
Epoch 109/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0223 - acc: 1.0000
Epoch 110/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0185 - acc: 1.0000
Epoch 111/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0184 - acc: 1.0000
Epoch 112/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0183 - acc: 1.0000
Epoch 113/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0181 - acc: 1.0000
Epoch 114/150
15/15 [===================

In [132]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.3746
Score O2 avg: 0.3654


In [133]:
model = cnn5(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
15/15 [==============================] - 1s 70ms/step - loss: 2.0623 - acc: 0.2000
Epoch 2/50
15/15 [==============================] - 0s 2ms/step - loss: 1.7569 - acc: 0.1333
Epoch 3/50
15/15 [==============================] - 0s 2ms/step - loss: 1.7375 - acc: 0.2667
Epoch 4/50
15/15 [==============================] - 0s 2ms/step - loss: 1.7063 - acc: 0.1333
Epoch 5/50
15/15 [==============================] - 0s 1ms/step - loss: 1.7438 - acc: 0.0667
Epoch 6/50
15/15 [==============================] - 0s 1ms/step - loss: 1.6670 - acc: 0.2000
Epoch 7/50
15/15 [==============================] - 0s 1ms/step - loss: 1.6337 - acc: 0.2000
Epoch 8/50
15/15 [==============================] - 0s 1ms/step - loss: 1.5833 - acc: 0.4000
Epoch 9/50
15/15 [==============================] - 0s 1ms/step - loss: 1.6641 - acc: 0.3333
Epoch 10/50
15/15 [==============================] - 0s 1ms/step - loss: 1.5800 - acc: 0.2667
Epoch 11/50
15/15 [==============================] - 0s 2ms

Epoch 38/150
15/15 [==============================] - 0s 2ms/step - loss: 0.6573 - acc: 0.8000
Epoch 39/150
15/15 [==============================] - 0s 1ms/step - loss: 0.8550 - acc: 0.7333
Epoch 40/150
15/15 [==============================] - 0s 1ms/step - loss: 0.9550 - acc: 0.5333
Epoch 41/150
15/15 [==============================] - 0s 1ms/step - loss: 0.7150 - acc: 0.7333
Epoch 42/150
15/15 [==============================] - 0s 1ms/step - loss: 0.8670 - acc: 0.6000
Epoch 43/150
15/15 [==============================] - 0s 1ms/step - loss: 0.8821 - acc: 0.7333
Epoch 44/150
15/15 [==============================] - 0s 1ms/step - loss: 0.7796 - acc: 0.8667
Epoch 45/150
15/15 [==============================] - 0s 1ms/step - loss: 0.8874 - acc: 0.6667
Epoch 46/150
15/15 [==============================] - 0s 1ms/step - loss: 0.7721 - acc: 0.6667
Epoch 47/150
15/15 [==============================] - 0s 1ms/step - loss: 0.8535 - acc: 0.6000
Epoch 48/150
15/15 [==============================

Epoch 124/150
15/15 [==============================] - 0s 1ms/step - loss: 0.5272 - acc: 0.8000
Epoch 125/150
15/15 [==============================] - 0s 1ms/step - loss: 0.5608 - acc: 0.8667
Epoch 126/150
15/15 [==============================] - 0s 2ms/step - loss: 0.5815 - acc: 0.8667
Epoch 127/150
15/15 [==============================] - 0s 1ms/step - loss: 0.5413 - acc: 0.8000
Epoch 128/150
15/15 [==============================] - 0s 1ms/step - loss: 0.7094 - acc: 0.7333
Epoch 129/150
15/15 [==============================] - 0s 1ms/step - loss: 0.5644 - acc: 0.8000
Epoch 130/150
15/15 [==============================] - 0s 1ms/step - loss: 0.5271 - acc: 0.9333
Epoch 131/150
15/15 [==============================] - 0s 1ms/step - loss: 0.5263 - acc: 0.8000
Epoch 132/150
15/15 [==============================] - 0s 1ms/step - loss: 0.4349 - acc: 0.9333
Epoch 133/150
15/15 [==============================] - 0s 1ms/step - loss: 0.5155 - acc: 0.8000
Epoch 134/150
15/15 [===================

Epoch 11/150
15/15 [==============================] - 0s 1ms/step - loss: 0.4344 - acc: 0.8667
Epoch 12/150
15/15 [==============================] - 0s 1ms/step - loss: 0.4926 - acc: 0.8667
Epoch 13/150
15/15 [==============================] - 0s 1ms/step - loss: 0.8075 - acc: 0.7333
Epoch 14/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3562 - acc: 1.0000
Epoch 15/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3845 - acc: 0.8000
Epoch 16/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3838 - acc: 0.9333
Epoch 17/150
15/15 [==============================] - 0s 2ms/step - loss: 0.4987 - acc: 0.8000
Epoch 18/150
15/15 [==============================] - 0s 1ms/step - loss: 0.4291 - acc: 0.8667
Epoch 19/150
15/15 [==============================] - 0s 1ms/step - loss: 0.5303 - acc: 0.8000
Epoch 20/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3293 - acc: 0.9333
Epoch 21/150
15/15 [==============================

Epoch 98/150
15/15 [==============================] - 0s 2ms/step - loss: 0.3448 - acc: 0.9333
Epoch 99/150
15/15 [==============================] - 0s 1ms/step - loss: 0.4877 - acc: 0.8000
Epoch 100/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3689 - acc: 0.8667
Epoch 101/150
15/15 [==============================] - 0s 1ms/step - loss: 0.4390 - acc: 0.8667
Epoch 102/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2759 - acc: 0.9333
Epoch 103/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3472 - acc: 0.8667
Epoch 104/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2939 - acc: 0.9333
Epoch 105/150
15/15 [==============================] - 0s 1ms/step - loss: 0.4719 - acc: 0.8667
Epoch 106/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1652 - acc: 1.0000
Epoch 107/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3253 - acc: 0.8667
Epoch 108/150
15/15 [=====================

...

Epoch 7/50
15/15 [==============================] - 0s 1ms/step - loss: 0.1141 - acc: 1.0000
Epoch 8/50
15/15 [==============================] - 0s 2ms/step - loss: 0.2676 - acc: 0.9333
Epoch 9/50
15/15 [==============================] - 0s 1ms/step - loss: 0.2080 - acc: 1.0000
Epoch 10/50
15/15 [==============================] - 0s 2ms/step - loss: 0.1878 - acc: 0.9333
Epoch 11/50
15/15 [==============================] - 0s 2ms/step - loss: 0.3672 - acc: 1.0000
Epoch 12/50
15/15 [==============================] - 0s 2ms/step - loss: 0.1520 - acc: 1.0000
Epoch 13/50
15/15 [==============================] - 0s 2ms/step - loss: 0.4450 - acc: 0.8000
Epoch 14/50
15/15 [==============================] - 0s 1ms/step - loss: 0.3777 - acc: 0.9333
Epoch 15/50
15/15 [==============================] - 0s 2ms/step - loss: 0.1949 - acc: 0.9333
Epoch 16/50
15/15 [==============================] - 0s 2ms/step - loss: 0.2258 - acc: 0.9333
Epoch 17/50
15/15 [==============================] - 0s 1ms/ste

Epoch 44/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2006 - acc: 0.9333
Epoch 45/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1377 - acc: 1.0000
Epoch 46/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3998 - acc: 0.8000
Epoch 47/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1925 - acc: 0.9333
Epoch 48/150
15/15 [==============================] - 0s 2ms/step - loss: 0.2231 - acc: 0.9333
Epoch 49/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2217 - acc: 0.9333
Epoch 50/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1821 - acc: 0.9333
Epoch 51/150
15/15 [==============================] - 0s 2ms/step - loss: 0.2072 - acc: 1.0000
Epoch 52/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1485 - acc: 0.9333
Epoch 53/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3969 - acc: 0.8667
Epoch 54/150
15/15 [==============================

Epoch 130/150
15/15 [==============================] - 0s 2ms/step - loss: 0.2174 - acc: 0.9333
Epoch 131/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2950 - acc: 0.9333
Epoch 132/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0873 - acc: 1.0000
Epoch 133/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1801 - acc: 0.9333
Epoch 134/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1033 - acc: 1.0000
Epoch 135/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3327 - acc: 0.8667
Epoch 136/150
15/15 [==============================] - 0s 1ms/step - loss: 0.5803 - acc: 0.7333
Epoch 137/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1293 - acc: 1.0000
Epoch 138/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3318 - acc: 0.8000
Epoch 139/150
15/15 [==============================] - 0s 1ms/step - loss: 0.4546 - acc: 0.8667
Epoch 140/150
15/15 [===================

Epoch 17/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2134 - acc: 1.0000
Epoch 18/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1161 - acc: 1.0000
Epoch 19/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2596 - acc: 0.8667
Epoch 20/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2096 - acc: 0.9333
Epoch 21/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1922 - acc: 1.0000
Epoch 22/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1833 - acc: 0.9333
Epoch 23/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2403 - acc: 0.9333
Epoch 24/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1407 - acc: 1.0000
Epoch 25/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1107 - acc: 1.0000
Epoch 26/150
15/15 [==============================] - 0s 2ms/step - loss: 0.2070 - acc: 0.9333
Epoch 27/150
15/15 [==============================

Epoch 104/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2415 - acc: 0.9333
Epoch 105/150
15/15 [==============================] - 0s 1ms/step - loss: 0.3662 - acc: 0.8000
Epoch 106/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0793 - acc: 1.0000
Epoch 107/150
15/15 [==============================] - 0s 2ms/step - loss: 0.0943 - acc: 1.0000
Epoch 108/150
15/15 [==============================] - 0s 1ms/step - loss: 0.0872 - acc: 1.0000
Epoch 109/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1821 - acc: 0.9333
Epoch 110/150
15/15 [==============================] - 0s 1ms/step - loss: 0.1097 - acc: 1.0000
Epoch 111/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1730 - acc: 0.9333
Epoch 112/150
15/15 [==============================] - 0s 2ms/step - loss: 0.1867 - acc: 0.9333
Epoch 113/150
15/15 [==============================] - 0s 1ms/step - loss: 0.2228 - acc: 0.8667
Epoch 114/150
15/15 [===================

In [134]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.3292
Score O2 avg: 0.3444


In [135]:
nb_categories = 10
nb_exemplars = 3
X_train, Y_train, X_test_o1, X_test_o2 = get_data(nb_categories, nb_exemplars)

In [136]:
model = cnn4(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
30/30 [==============================] - 1s 48ms/step - loss: 2.7172 - acc: 0.0667
Epoch 2/50
30/30 [==============================] - 0s 2ms/step - loss: 2.4039 - acc: 0.1333
Epoch 3/50
30/30 [==============================] - 0s 2ms/step - loss: 2.3537 - acc: 0.1667
Epoch 4/50
30/30 [==============================] - 0s 2ms/step - loss: 2.3545 - acc: 0.1000
Epoch 5/50
30/30 [==============================] - 0s 1ms/step - loss: 2.4170 - acc: 0.0667
Epoch 6/50
30/30 [==============================] - 0s 1ms/step - loss: 2.2558 - acc: 0.3667
Epoch 7/50
30/30 [==============================] - 0s 1ms/step - loss: 2.3365 - acc: 0.3000
Epoch 8/50
30/30 [==============================] - 0s 1ms/step - loss: 2.2035 - acc: 0.2333
Epoch 9/50
30/30 [==============================] - 0s 1ms/step - loss: 2.2392 - acc: 0.3000
Epoch 10/50
30/30 [==============================] - 0s 1ms/step - loss: 2.2805 - acc: 0.3667
Epoch 11/50
30/30 [==============================] - 0s 1ms

Epoch 38/150
30/30 [==============================] - 0s 2ms/step - loss: 0.3237 - acc: 1.0000
Epoch 39/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3871 - acc: 0.9667
Epoch 40/150
30/30 [==============================] - 0s 2ms/step - loss: 0.2535 - acc: 1.0000
Epoch 41/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4151 - acc: 0.8667
Epoch 42/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4102 - acc: 0.8667
Epoch 43/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3360 - acc: 0.9667
Epoch 44/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3807 - acc: 0.9333
Epoch 45/150
30/30 [==============================] - 0s 1ms/step - loss: 0.2716 - acc: 1.0000
Epoch 46/150
30/30 [==============================] - 0s 1ms/step - loss: 0.2629 - acc: 0.9667
Epoch 47/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3281 - acc: 0.9333
Epoch 48/150
30/30 [==============================

Epoch 124/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1650 - acc: 0.9667
Epoch 125/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1385 - acc: 1.0000
Epoch 126/150
30/30 [==============================] - 0s 2ms/step - loss: 0.1189 - acc: 1.0000
Epoch 127/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1619 - acc: 0.9667
Epoch 128/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1981 - acc: 0.9667
Epoch 129/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1487 - acc: 0.9667
Epoch 130/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1511 - acc: 0.9667
Epoch 131/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1829 - acc: 0.9667
Epoch 132/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1781 - acc: 0.9667
Epoch 133/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1420 - acc: 0.9667
Epoch 134/150
30/30 [===================

Epoch 11/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0880 - acc: 1.0000
Epoch 12/150
30/30 [==============================] - 0s 2ms/step - loss: 0.0910 - acc: 1.0000
Epoch 13/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0943 - acc: 1.0000
Epoch 14/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1113 - acc: 1.0000
Epoch 15/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0963 - acc: 1.0000
Epoch 16/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0965 - acc: 1.0000
Epoch 17/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0944 - acc: 1.0000
Epoch 18/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0954 - acc: 1.0000
Epoch 19/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1269 - acc: 1.0000
Epoch 20/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1220 - acc: 1.0000
Epoch 21/150
30/30 [==============================

Epoch 98/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0963 - acc: 1.0000
Epoch 99/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0920 - acc: 1.0000
Epoch 100/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0960 - acc: 1.0000
Epoch 101/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1094 - acc: 1.0000
Epoch 102/150
30/30 [==============================] - 0s 2ms/step - loss: 0.0827 - acc: 1.0000
Epoch 103/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1385 - acc: 1.0000
Epoch 104/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0848 - acc: 1.0000
Epoch 105/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0835 - acc: 1.0000
Epoch 106/150
30/30 [==============================] - 0s 2ms/step - loss: 0.0894 - acc: 1.0000
Epoch 107/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0843 - acc: 1.0000
Epoch 108/150
30/30 [=====================

...

Epoch 7/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0670 - acc: 1.0000
Epoch 8/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0604 - acc: 1.0000
Epoch 9/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0759 - acc: 1.0000
Epoch 10/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0661 - acc: 1.0000
Epoch 11/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0730 - acc: 1.0000
Epoch 12/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0831 - acc: 1.0000
Epoch 13/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0607 - acc: 1.0000
Epoch 14/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0598 - acc: 1.0000
Epoch 15/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0597 - acc: 1.0000
Epoch 16/50
30/30 [==============================] - 0s 1ms/step - loss: 0.0597 - acc: 1.0000
Epoch 17/50
30/30 [==============================] - 0s 1ms/ste

Epoch 44/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0563 - acc: 1.0000
Epoch 45/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0543 - acc: 1.0000
Epoch 46/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0670 - acc: 1.0000
Epoch 47/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0538 - acc: 1.0000
Epoch 48/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0641 - acc: 1.0000
Epoch 49/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0720 - acc: 1.0000
Epoch 50/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0555 - acc: 1.0000
Epoch 51/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0565 - acc: 1.0000
Epoch 52/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0617 - acc: 1.0000
Epoch 53/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0583 - acc: 1.0000
Epoch 54/150
30/30 [==============================

Epoch 130/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0504 - acc: 1.0000
Epoch 131/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0526 - acc: 1.0000
Epoch 132/150
30/30 [==============================] - 0s 2ms/step - loss: 0.0470 - acc: 1.0000
Epoch 133/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0474 - acc: 1.0000
Epoch 134/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0480 - acc: 1.0000
Epoch 135/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0472 - acc: 1.0000
Epoch 136/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0491 - acc: 1.0000
Epoch 137/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0470 - acc: 1.0000
Epoch 138/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0523 - acc: 1.0000
Epoch 139/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0459 - acc: 1.0000
Epoch 140/150
30/30 [===================

Epoch 17/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0696 - acc: 0.9667
Epoch 18/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1236 - acc: 1.0000
Epoch 19/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0650 - acc: 1.0000
Epoch 20/150
30/30 [==============================] - 0s 2ms/step - loss: 0.0405 - acc: 1.0000
Epoch 21/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0462 - acc: 1.0000
Epoch 22/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0793 - acc: 0.9667
Epoch 23/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0464 - acc: 1.0000
Epoch 24/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0424 - acc: 1.0000
Epoch 25/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0438 - acc: 1.0000
Epoch 26/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0761 - acc: 1.0000
Epoch 27/150
30/30 [==============================

Epoch 104/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0366 - acc: 1.0000
Epoch 105/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0420 - acc: 1.0000
Epoch 106/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0415 - acc: 1.0000
Epoch 107/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0377 - acc: 1.0000
Epoch 108/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0438 - acc: 1.0000
Epoch 109/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0401 - acc: 1.0000
Epoch 110/150
30/30 [==============================] - 0s 2ms/step - loss: 0.0360 - acc: 1.0000
Epoch 111/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0383 - acc: 1.0000
Epoch 112/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0385 - acc: 1.0000
Epoch 113/150
30/30 [==============================] - 0s 1ms/step - loss: 0.0363 - acc: 1.0000
Epoch 114/150
30/30 [===================

In [137]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.6590
Score O2 avg: 0.5198


In [138]:
model = cnn5(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
30/30 [==============================] - 1s 46ms/step - loss: 4.9453 - acc: 0.1333
Epoch 2/50
30/30 [==============================] - 0s 1ms/step - loss: 2.7944 - acc: 0.1000
Epoch 3/50
30/30 [==============================] - 0s 1ms/step - loss: 2.5443 - acc: 0.1000
Epoch 4/50
30/30 [==============================] - 0s 1ms/step - loss: 2.4419 - acc: 0.0667
Epoch 5/50
30/30 [==============================] - 0s 1ms/step - loss: 2.4527 - acc: 0.1333
Epoch 6/50
30/30 [==============================] - 0s 1ms/step - loss: 2.5846 - acc: 0.0333
Epoch 7/50
30/30 [==============================] - 0s 2ms/step - loss: 2.4738 - acc: 0.1000
Epoch 8/50
30/30 [==============================] - 0s 1ms/step - loss: 2.4026 - acc: 0.1333
Epoch 9/50
30/30 [==============================] - 0s 1ms/step - loss: 2.3698 - acc: 0.2000
Epoch 10/50
30/30 [==============================] - 0s 1ms/step - loss: 2.3728 - acc: 0.0667
Epoch 11/50
30/30 [==============================] - 0s 1ms

Epoch 38/150
30/30 [==============================] - 0s 1ms/step - loss: 1.4124 - acc: 0.4667
Epoch 39/150
30/30 [==============================] - 0s 1ms/step - loss: 1.4573 - acc: 0.6000
Epoch 40/150
30/30 [==============================] - 0s 1ms/step - loss: 1.5051 - acc: 0.4667
Epoch 41/150
30/30 [==============================] - 0s 1ms/step - loss: 1.5085 - acc: 0.4333
Epoch 42/150
30/30 [==============================] - 0s 1ms/step - loss: 1.4452 - acc: 0.6000
Epoch 43/150
30/30 [==============================] - 0s 1ms/step - loss: 1.4598 - acc: 0.4667
Epoch 44/150
30/30 [==============================] - 0s 1ms/step - loss: 1.4441 - acc: 0.5000
Epoch 45/150
30/30 [==============================] - 0s 1ms/step - loss: 1.2825 - acc: 0.6667
Epoch 46/150
30/30 [==============================] - 0s 2ms/step - loss: 1.0562 - acc: 0.6000
Epoch 47/150
30/30 [==============================] - 0s 1ms/step - loss: 1.3063 - acc: 0.5333
Epoch 48/150
30/30 [==============================

Epoch 124/150
30/30 [==============================] - 0s 2ms/step - loss: 0.6710 - acc: 0.7667
Epoch 125/150
30/30 [==============================] - 0s 1ms/step - loss: 0.7834 - acc: 0.8333
Epoch 126/150
30/30 [==============================] - 0s 1ms/step - loss: 0.8545 - acc: 0.6667
Epoch 127/150
30/30 [==============================] - 0s 1ms/step - loss: 0.8413 - acc: 0.7333
Epoch 128/150
30/30 [==============================] - 0s 1ms/step - loss: 0.8301 - acc: 0.7667
Epoch 129/150
30/30 [==============================] - 0s 1ms/step - loss: 0.7689 - acc: 0.8000
Epoch 130/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6837 - acc: 0.7333
Epoch 131/150
30/30 [==============================] - 0s 1ms/step - loss: 1.0699 - acc: 0.5667
Epoch 132/150
30/30 [==============================] - 0s 1ms/step - loss: 0.8762 - acc: 0.7333
Epoch 133/150
30/30 [==============================] - 0s 1ms/step - loss: 0.7036 - acc: 0.7000
Epoch 134/150
30/30 [===================

Epoch 11/150
30/30 [==============================] - 0s 1ms/step - loss: 0.8161 - acc: 0.7000
Epoch 12/150
30/30 [==============================] - 0s 1ms/step - loss: 0.7548 - acc: 0.7333
Epoch 13/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6703 - acc: 0.9000
Epoch 14/150
30/30 [==============================] - 0s 2ms/step - loss: 0.5858 - acc: 0.7667
Epoch 15/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6414 - acc: 0.8000
Epoch 16/150
30/30 [==============================] - 0s 2ms/step - loss: 0.5176 - acc: 0.8333
Epoch 17/150
30/30 [==============================] - 0s 1ms/step - loss: 0.9354 - acc: 0.6333
Epoch 18/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6274 - acc: 0.7667
Epoch 19/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5980 - acc: 0.7667
Epoch 20/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6678 - acc: 0.7667
Epoch 21/150
30/30 [==============================

Epoch 98/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6295 - acc: 0.8000
Epoch 99/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5915 - acc: 0.8000
Epoch 100/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6657 - acc: 0.8333
Epoch 101/150
30/30 [==============================] - 0s 1ms/step - loss: 0.7309 - acc: 0.7333
Epoch 102/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5212 - acc: 0.8667
Epoch 103/150
30/30 [==============================] - 0s 1ms/step - loss: 0.7012 - acc: 0.7667
Epoch 104/150
30/30 [==============================] - 0s 1ms/step - loss: 0.8310 - acc: 0.6333
Epoch 105/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5868 - acc: 0.8000
Epoch 106/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6182 - acc: 0.8000
Epoch 107/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6060 - acc: 0.7667
Epoch 108/150
30/30 [=====================

...

Epoch 7/50
30/30 [==============================] - 0s 1ms/step - loss: 0.6482 - acc: 0.7667
Epoch 8/50
30/30 [==============================] - 0s 1ms/step - loss: 0.6476 - acc: 0.8333
Epoch 9/50
30/30 [==============================] - 0s 1ms/step - loss: 0.5332 - acc: 0.8333
Epoch 10/50
30/30 [==============================] - 0s 1ms/step - loss: 0.4276 - acc: 0.8667
Epoch 11/50
30/30 [==============================] - 0s 1ms/step - loss: 0.5038 - acc: 0.8333
Epoch 12/50
30/30 [==============================] - 0s 1ms/step - loss: 0.6420 - acc: 0.7667
Epoch 13/50
30/30 [==============================] - 0s 1ms/step - loss: 0.5747 - acc: 0.8667
Epoch 14/50
30/30 [==============================] - 0s 1ms/step - loss: 0.4469 - acc: 0.8333
Epoch 15/50
30/30 [==============================] - 0s 1ms/step - loss: 0.4809 - acc: 0.8333
Epoch 16/50
30/30 [==============================] - 0s 1ms/step - loss: 0.9400 - acc: 0.6000
Epoch 17/50
30/30 [==============================] - 0s 1ms/ste

Epoch 44/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4901 - acc: 0.9000
Epoch 45/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6629 - acc: 0.7667
Epoch 46/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5037 - acc: 0.8667
Epoch 47/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3963 - acc: 0.8667
Epoch 48/150
30/30 [==============================] - 0s 1ms/step - loss: 0.2622 - acc: 0.9333
Epoch 49/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3499 - acc: 0.8667
Epoch 50/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5244 - acc: 0.8000
Epoch 51/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5887 - acc: 0.7667
Epoch 52/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5659 - acc: 0.7333
Epoch 53/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6583 - acc: 0.7333
Epoch 54/150
30/30 [==============================

Epoch 130/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6223 - acc: 0.8000
Epoch 131/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4689 - acc: 0.9000
Epoch 132/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4330 - acc: 0.8333
Epoch 133/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4891 - acc: 0.8000
Epoch 134/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3635 - acc: 0.9000
Epoch 135/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3833 - acc: 0.8667
Epoch 136/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5997 - acc: 0.7667
Epoch 137/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5543 - acc: 0.7667
Epoch 138/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3931 - acc: 0.9000
Epoch 139/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6673 - acc: 0.7667
Epoch 140/150
30/30 [===================

Epoch 17/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3918 - acc: 0.9000
Epoch 18/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3775 - acc: 0.8667
Epoch 19/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4407 - acc: 0.8000
Epoch 20/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3166 - acc: 0.9000
Epoch 21/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5802 - acc: 0.8667
Epoch 22/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4265 - acc: 0.8667
Epoch 23/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6010 - acc: 0.8000
Epoch 24/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4787 - acc: 0.7667
Epoch 25/150
30/30 [==============================] - 0s 1ms/step - loss: 0.2950 - acc: 0.9000
Epoch 26/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5626 - acc: 0.8000
Epoch 27/150
30/30 [==============================

Epoch 104/150
30/30 [==============================] - 0s 1ms/step - loss: 0.1947 - acc: 0.9333
Epoch 105/150
30/30 [==============================] - 0s 1ms/step - loss: 0.2713 - acc: 1.0000
Epoch 106/150
30/30 [==============================] - 0s 1ms/step - loss: 0.2784 - acc: 0.9333
Epoch 107/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4179 - acc: 0.8667
Epoch 108/150
30/30 [==============================] - 0s 1ms/step - loss: 0.2392 - acc: 0.9667
Epoch 109/150
30/30 [==============================] - 0s 1ms/step - loss: 0.6266 - acc: 0.8000
Epoch 110/150
30/30 [==============================] - 0s 1ms/step - loss: 0.5374 - acc: 0.7333
Epoch 111/150
30/30 [==============================] - 0s 1ms/step - loss: 0.4846 - acc: 0.8000
Epoch 112/150
30/30 [==============================] - 0s 1ms/step - loss: 0.3207 - acc: 0.9000
Epoch 113/150
30/30 [==============================] - 0s 1ms/step - loss: 0.7167 - acc: 0.7000
Epoch 114/150
30/30 [===================

In [139]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.7212
Score O2 avg: 0.6244


In [140]:
nb_categories = 5
nb_exemplars = 5
X_train, Y_train, X_test_o1, X_test_o2 = get_data(nb_categories, nb_exemplars)

In [141]:
model = cnn4(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
25/25 [==============================] - 2s 62ms/step - loss: 1.9444 - acc: 0.2400
Epoch 2/50
25/25 [==============================] - 0s 2ms/step - loss: 1.5972 - acc: 0.2800
Epoch 3/50
25/25 [==============================] - 0s 1ms/step - loss: 1.5579 - acc: 0.4000
Epoch 4/50
25/25 [==============================] - 0s 1ms/step - loss: 1.5383 - acc: 0.3200
Epoch 5/50
25/25 [==============================] - 0s 1ms/step - loss: 1.5858 - acc: 0.3200
Epoch 6/50
25/25 [==============================] - 0s 1ms/step - loss: 1.4754 - acc: 0.4800
Epoch 7/50
25/25 [==============================] - 0s 1ms/step - loss: 1.5336 - acc: 0.4000
Epoch 8/50
25/25 [==============================] - 0s 1ms/step - loss: 1.3440 - acc: 0.5600
Epoch 9/50
25/25 [==============================] - 0s 1ms/step - loss: 1.2169 - acc: 0.6400
Epoch 10/50
25/25 [==============================] - 0s 1ms/step - loss: 1.2551 - acc: 0.6000
Epoch 11/50
25/25 [==============================] - 0s 1ms

Epoch 38/150
25/25 [==============================] - 0s 2ms/step - loss: 0.1394 - acc: 1.0000
Epoch 39/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1356 - acc: 1.0000
Epoch 40/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1771 - acc: 1.0000
Epoch 41/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2450 - acc: 0.9600
Epoch 42/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2932 - acc: 0.9200
Epoch 43/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4430 - acc: 0.8400
Epoch 44/150
25/25 [==============================] - 0s 1ms/step - loss: 0.5909 - acc: 0.8400
Epoch 45/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2400 - acc: 1.0000
Epoch 46/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2154 - acc: 1.0000
Epoch 47/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2039 - acc: 1.0000
Epoch 48/150
25/25 [==============================

Epoch 124/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1160 - acc: 1.0000
Epoch 125/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1051 - acc: 1.0000
Epoch 126/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1067 - acc: 1.0000
Epoch 127/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1032 - acc: 1.0000
Epoch 128/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0962 - acc: 1.0000
Epoch 129/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0946 - acc: 1.0000
Epoch 130/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0919 - acc: 1.0000
Epoch 131/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1158 - acc: 1.0000
Epoch 132/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0950 - acc: 1.0000
Epoch 133/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1067 - acc: 1.0000
Epoch 134/150
25/25 [===================

Epoch 11/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0919 - acc: 1.0000
Epoch 12/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0822 - acc: 1.0000
Epoch 13/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0873 - acc: 1.0000
Epoch 14/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0838 - acc: 1.0000
Epoch 15/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1067 - acc: 1.0000
Epoch 16/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0829 - acc: 1.0000
Epoch 17/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0841 - acc: 1.0000
Epoch 18/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0860 - acc: 1.0000
Epoch 19/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0804 - acc: 1.0000
Epoch 20/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0925 - acc: 1.0000
Epoch 21/150
25/25 [==============================

Epoch 98/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0759 - acc: 1.0000
Epoch 99/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0699 - acc: 1.0000
Epoch 100/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0646 - acc: 1.0000
Epoch 101/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0731 - acc: 1.0000
Epoch 102/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0715 - acc: 1.0000
Epoch 103/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0656 - acc: 1.0000
Epoch 104/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0752 - acc: 1.0000
Epoch 105/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1008 - acc: 0.9600
Epoch 106/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0690 - acc: 1.0000
Epoch 107/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0716 - acc: 1.0000
Epoch 108/150
25/25 [=====================

...

Epoch 7/50
25/25 [==============================] - 0s 1ms/step - loss: 0.0340 - acc: 1.0000
Epoch 8/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0323 - acc: 1.0000
Epoch 9/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0341 - acc: 1.0000
Epoch 10/50
25/25 [==============================] - 0s 1ms/step - loss: 0.0318 - acc: 1.0000
Epoch 11/50
25/25 [==============================] - 0s 1ms/step - loss: 0.0322 - acc: 1.0000
Epoch 12/50
25/25 [==============================] - 0s 1ms/step - loss: 0.0511 - acc: 1.0000
Epoch 13/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0343 - acc: 1.0000
Epoch 14/50
25/25 [==============================] - 0s 1ms/step - loss: 0.0323 - acc: 1.0000
Epoch 15/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0379 - acc: 1.0000
Epoch 16/50
25/25 [==============================] - 0s 2ms/step - loss: 0.0629 - acc: 1.0000
Epoch 17/50
25/25 [==============================] - 0s 1ms/ste

Epoch 44/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0283 - acc: 1.0000
Epoch 45/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0287 - acc: 1.0000
Epoch 46/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0560 - acc: 1.0000
Epoch 47/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0352 - acc: 1.0000
Epoch 48/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0274 - acc: 1.0000
Epoch 49/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0276 - acc: 1.0000
Epoch 50/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0290 - acc: 1.0000
Epoch 51/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0281 - acc: 1.0000
Epoch 52/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0274 - acc: 1.0000
Epoch 53/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0282 - acc: 1.0000
Epoch 54/150
25/25 [==============================

Epoch 130/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0280 - acc: 1.0000
Epoch 131/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0235 - acc: 1.0000
Epoch 132/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0358 - acc: 1.0000
Epoch 133/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0257 - acc: 1.0000
Epoch 134/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0246 - acc: 1.0000
Epoch 135/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0243 - acc: 1.0000
Epoch 136/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0241 - acc: 1.0000
Epoch 137/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0234 - acc: 1.0000
Epoch 138/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0233 - acc: 1.0000
Epoch 139/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0243 - acc: 1.0000
Epoch 140/150
25/25 [===================

Epoch 17/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0235 - acc: 1.0000
Epoch 18/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0194 - acc: 1.0000
Epoch 19/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0237 - acc: 1.0000
Epoch 20/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0194 - acc: 1.0000
Epoch 21/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0191 - acc: 1.0000
Epoch 22/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0195 - acc: 1.0000
Epoch 23/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0210 - acc: 1.0000
Epoch 24/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0188 - acc: 1.0000
Epoch 25/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0187 - acc: 1.0000
Epoch 26/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0184 - acc: 1.0000
Epoch 27/150
25/25 [==============================

Epoch 104/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0154 - acc: 1.0000
Epoch 105/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0148 - acc: 1.0000
Epoch 106/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0152 - acc: 1.0000
Epoch 107/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0151 - acc: 1.0000
Epoch 108/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0149 - acc: 1.0000
Epoch 109/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0151 - acc: 1.0000
Epoch 110/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0161 - acc: 1.0000
Epoch 111/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0174 - acc: 1.0000
Epoch 112/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0150 - acc: 1.0000
Epoch 113/150
25/25 [==============================] - 0s 1ms/step - loss: 0.0160 - acc: 1.0000
Epoch 114/150
25/25 [===================

In [142]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.6328
Score O2 avg: 0.3956


In [143]:
model = cnn5(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

Trial #1
Epoch 1/50
25/25 [==============================] - 1s 48ms/step - loss: 1.7145 - acc: 0.2400
Epoch 2/50
25/25 [==============================] - 0s 1ms/step - loss: 1.6956 - acc: 0.2800
Epoch 3/50
25/25 [==============================] - 0s 1ms/step - loss: 1.6916 - acc: 0.2400
Epoch 4/50
25/25 [==============================] - 0s 1ms/step - loss: 1.7017 - acc: 0.2000
Epoch 5/50
25/25 [==============================] - 0s 1ms/step - loss: 1.6724 - acc: 0.2800
Epoch 6/50
25/25 [==============================] - 0s 1ms/step - loss: 1.7378 - acc: 0.1200
Epoch 7/50
25/25 [==============================] - 0s 1ms/step - loss: 1.6847 - acc: 0.2400
Epoch 8/50
25/25 [==============================] - 0s 1ms/step - loss: 1.7019 - acc: 0.1200
Epoch 9/50
25/25 [==============================] - 0s 1ms/step - loss: 1.6884 - acc: 0.2000
Epoch 10/50
25/25 [==============================] - 0s 1ms/step - loss: 1.6808 - acc: 0.2000
Epoch 11/50
25/25 [==============================] - 0s 1ms

Epoch 38/150
25/25 [==============================] - 0s 3ms/step - loss: 1.0276 - acc: 0.5600
Epoch 39/150
25/25 [==============================] - 0s 1ms/step - loss: 1.0354 - acc: 0.5600
Epoch 40/150
25/25 [==============================] - 0s 3ms/step - loss: 0.9896 - acc: 0.6400
Epoch 41/150
25/25 [==============================] - 0s 1ms/step - loss: 1.0363 - acc: 0.7200
Epoch 42/150
25/25 [==============================] - 0s 1ms/step - loss: 1.0164 - acc: 0.5600
Epoch 43/150
25/25 [==============================] - 0s 1ms/step - loss: 0.9800 - acc: 0.5600
Epoch 44/150
25/25 [==============================] - 0s 1ms/step - loss: 1.2336 - acc: 0.5200
Epoch 45/150
25/25 [==============================] - 0s 1ms/step - loss: 0.9818 - acc: 0.6400
Epoch 46/150
25/25 [==============================] - 0s 2ms/step - loss: 0.8466 - acc: 0.7200
Epoch 47/150
25/25 [==============================] - 0s 1ms/step - loss: 0.9169 - acc: 0.6000
Epoch 48/150
25/25 [==============================

Epoch 124/150
25/25 [==============================] - 0s 1ms/step - loss: 0.5210 - acc: 0.8800
Epoch 125/150
25/25 [==============================] - 0s 1ms/step - loss: 0.6183 - acc: 0.7200
Epoch 126/150
25/25 [==============================] - 0s 2ms/step - loss: 0.4815 - acc: 0.8000
Epoch 127/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4680 - acc: 0.8000
Epoch 128/150
25/25 [==============================] - 0s 1ms/step - loss: 0.7713 - acc: 0.6800
Epoch 129/150
25/25 [==============================] - 0s 1ms/step - loss: 0.6641 - acc: 0.7200
Epoch 130/150
25/25 [==============================] - 0s 1ms/step - loss: 0.6660 - acc: 0.6800
Epoch 131/150
25/25 [==============================] - 0s 1ms/step - loss: 0.5723 - acc: 0.8000
Epoch 132/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4323 - acc: 0.8400
Epoch 133/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4621 - acc: 0.7600
Epoch 134/150
25/25 [===================

Epoch 11/150
25/25 [==============================] - 0s 1ms/step - loss: 0.5768 - acc: 0.7600
Epoch 12/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2938 - acc: 0.8800
Epoch 13/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4239 - acc: 0.8000
Epoch 14/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3088 - acc: 0.9200
Epoch 15/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3098 - acc: 0.8800
Epoch 16/150
25/25 [==============================] - 0s 1ms/step - loss: 0.5220 - acc: 0.7600
Epoch 17/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4602 - acc: 0.8000
Epoch 18/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3556 - acc: 0.8800
Epoch 19/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3768 - acc: 0.8400
Epoch 20/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4965 - acc: 0.7600
Epoch 21/150
25/25 [==============================

Epoch 98/150
25/25 [==============================] - 0s 2ms/step - loss: 0.3119 - acc: 0.8400
Epoch 99/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2440 - acc: 0.9200
Epoch 100/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2237 - acc: 0.9200
Epoch 101/150
25/25 [==============================] - 0s 1ms/step - loss: 0.5228 - acc: 0.8000
Epoch 102/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3461 - acc: 0.8400
Epoch 103/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4924 - acc: 0.8000
Epoch 104/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2729 - acc: 0.9600
Epoch 105/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2944 - acc: 0.8800
Epoch 106/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2947 - acc: 0.9200
Epoch 107/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4108 - acc: 0.8400
Epoch 108/150
25/25 [=====================

...

Epoch 7/50
25/25 [==============================] - 0s 1ms/step - loss: 0.2887 - acc: 0.8800
Epoch 8/50
25/25 [==============================] - 0s 1ms/step - loss: 0.3567 - acc: 0.8000
Epoch 9/50
25/25 [==============================] - 0s 1ms/step - loss: 0.3351 - acc: 0.9200
Epoch 10/50
25/25 [==============================] - 0s 1ms/step - loss: 0.3144 - acc: 0.9200
Epoch 11/50
25/25 [==============================] - 0s 1ms/step - loss: 0.1751 - acc: 0.9600
Epoch 12/50
25/25 [==============================] - 0s 1ms/step - loss: 0.2755 - acc: 0.8800
Epoch 13/50
25/25 [==============================] - 0s 1ms/step - loss: 0.3624 - acc: 0.8800
Epoch 14/50
25/25 [==============================] - 0s 1ms/step - loss: 0.2981 - acc: 0.8400
Epoch 15/50
25/25 [==============================] - 0s 1ms/step - loss: 0.3635 - acc: 0.8400
Epoch 16/50
25/25 [==============================] - 0s 1ms/step - loss: 0.3126 - acc: 0.8800
Epoch 17/50
25/25 [==============================] - 0s 1ms/ste

Epoch 44/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2917 - acc: 0.8800
Epoch 45/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1548 - acc: 0.9600
Epoch 46/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2046 - acc: 0.9200
Epoch 47/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1447 - acc: 1.0000
Epoch 48/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2293 - acc: 0.9200
Epoch 49/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1881 - acc: 0.9600
Epoch 50/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1456 - acc: 0.9600
Epoch 51/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3224 - acc: 0.8800
Epoch 52/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2820 - acc: 0.8800
Epoch 53/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3229 - acc: 0.8800
Epoch 54/150
25/25 [==============================

Epoch 130/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1965 - acc: 0.9600
Epoch 131/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2468 - acc: 0.9200
Epoch 132/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2548 - acc: 0.8800
Epoch 133/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1763 - acc: 0.9600
Epoch 134/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2822 - acc: 0.8800
Epoch 135/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3569 - acc: 0.9200
Epoch 136/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3046 - acc: 0.8800
Epoch 137/150
25/25 [==============================] - 0s 2ms/step - loss: 0.1899 - acc: 0.9600
Epoch 138/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3840 - acc: 0.8800
Epoch 139/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2652 - acc: 0.9200
Epoch 140/150
25/25 [===================

Epoch 17/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2541 - acc: 0.9200
Epoch 18/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1794 - acc: 0.9600
Epoch 19/150
25/25 [==============================] - 0s 2ms/step - loss: 0.3566 - acc: 0.8400
Epoch 20/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1856 - acc: 0.9200
Epoch 21/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2219 - acc: 0.8800
Epoch 22/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1397 - acc: 1.0000
Epoch 23/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2085 - acc: 0.9200
Epoch 24/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2275 - acc: 0.8800
Epoch 25/150
25/25 [==============================] - 0s 1ms/step - loss: 0.4897 - acc: 0.7600
Epoch 26/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1576 - acc: 0.9600
Epoch 27/150
25/25 [==============================

Epoch 104/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2973 - acc: 0.8800
Epoch 105/150
25/25 [==============================] - 0s 1ms/step - loss: 0.1326 - acc: 0.9600
Epoch 106/150
25/25 [==============================] - 0s 2ms/step - loss: 0.2053 - acc: 0.9200
Epoch 107/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2522 - acc: 0.8800
Epoch 108/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2049 - acc: 0.9200
Epoch 109/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2972 - acc: 0.8800
Epoch 110/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3226 - acc: 0.8400
Epoch 111/150
25/25 [==============================] - 0s 1ms/step - loss: 0.3861 - acc: 0.8000
Epoch 112/150
25/25 [==============================] - 0s 2ms/step - loss: 0.0675 - acc: 1.0000
Epoch 113/150
25/25 [==============================] - 0s 1ms/step - loss: 0.2093 - acc: 0.9200
Epoch 114/150
25/25 [===================

In [144]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

Score O1 avg: 0.8372
Score O2 avg: 0.6574


In [ ]:
nb_categories = 50
nb_exemplars = 15
X_train, Y_train, X_test_o1, X_test_o2 = get_data(nb_categories, nb_exemplars)

In [ ]:
model = cnn4(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

In [ ]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())

In [ ]:
model = cnn5(
    input_shape=X_train.shape[1:], 
    nb_classes=Y_train.shape[-1]
)
scores_o1, scores_o2 = model_trial(
    model, X_train, Y_train, X_test_o1, X_test_o2, 
    nb_epoch=200, burn_period=50
)

In [ ]:
print('Score O1 avg: %0.4f' % scores_o1.mean())
print('Score O2 avg: %0.4f' % scores_o2.mean())